# **02_data_api_processing_grouping.ipynb**
В этом ноутбуке выполняется обогащение собранных данных о недвижимомти Москвы с помощью дополнительной информации из API 2ГИС.

Через API получены данные об инфраструктуре рядом с объектами недвижимости (школы, детские сады, клиники, кафе, парки, бизнес-центры и др.).

In [ ]:
import requests
import json
import time
from typing import List, Dict, Any
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import numpy as np

------------------------------------------------------------------------------------------------------------------------------------------------------

Находим id районов Москвы (для дальнейших запросов при парсинге). (id определенного района находится в ссылке при открытии странички про этот район на 2гис)

In [ ]:
import requests
import time

api_key = "f50ee13f-107c-47c0-bae5-bf3feb6750bc"

MOSCOW_DISTRICTS = [
    "Академический", "Алексеевский", "Алтуфьевский", "Арбат", "Аэропорт",
    "Бабушкинский", "Басманный", "Беговой", "Бекасово", "Бескудниковский",
    "Бибирево", "Бирюлёво Восточное", "Бирюлёво Западное", "Богородское",
    "Братеево", "Бутырский", "Вешняки", "Внуково", "Войковский", "Вороново",
    "Восточное Дегунино", "Восточное Измайлово", "Восточный", "Выхино-Жулебино",
    "Гагаринский", "Головинский", "Гольяново", "Даниловский", "Дмитровский",
    "Донской", "Дорогомилово", "Замоскворечье", "Западное Дегунино", "Зюзино",
    "Зябликово", "Ивановское", "Измайлово", "Капотня", "Коммунарка", "Коньково",
    "Коптево", "Косино-Ухтомский", "Котловка", "Краснопахорский", "Красносельский",
    "Крылатское", "Крюково", "Кузьминки", "Кунцево", "Куркино", "Левобережный",
    "Лефортово", "Лианозово", "Ломоносовский", "Лосиноостровский", "Люблино",
    "Марфино", "Марьина Роща", "Марьино", "Матушкино", "Метрогородок",
    "Мещанский", "Митино", "Можайский", "Молжаниновский", "Москворечье-Сабурово",
    "Нагатино-Садовники", "Нагатинский Затон", "Нагорный", "Некрасовка",
    "Нижегородский", "Новогиреево", "Новокосино", "Ново-Переделкино",
    "Обручевский", "Орехово-Борисово Северное", "Орехово-Борисово Южное",
    "Останкинский", "Отрадное", "Очаково-Матвеевское", "Перово", "Печатники",
    "Покровское-Стрешнево", "Преображенское", "Пресненский",
    "Проспект Вернадского", "Раменки", "Ростокино", "Рязанский", "Савёлки",
    "Савёловский", "Свиблово", "Северное Бутово", "Северное Измайлово",
    "Северное Медведково", "Северное Тушино", "Северный", "Силино", "Сокол",
    "Соколиная Гора", "Сокольники", "Солнцево", "Старое Крюково", "Строгино",
    "Таганский", "Тверской", "Текстильщики", "Тёплый Стан", "Тимирязевский",
    "Троицк", "Тропарёво-Никулино", "Филёвский Парк", "Фили-Давыдково",
    "Филимонковский", "Хамовники", "Ховрино", "Хорошёво-Мнёвники",
    "Хорошёвский", "Царицыно", "Черёмушки", "Чертаново Северное",
    "Чертаново Центральное", "Чертаново Южное", "Щербинка", "Щукино",
    "Южное Бутово", "Южное Медведково", "Южное Тушино", "Южнопортовый",
    "Якиманка", "Ярославский", "Ясенево"
]


def get_region_id(region_name):
    """Функция для получения ID района по его названию через API 2ГИС"""

    url = f"https://catalog.api.2gis.com/3.0/items?q={region_name} район Москва&key={api_key}&type=adm_div.district"

    try:
        response = requests.get(url)
        data = response.json()

        if 'result' in data and 'items' in data['result']:
            items = data['result']['items']
            if items: # Если есть результаты
                # Берем первый результат, так как это самый релевантный
                return items[0]['id']
            else:
                print(f"Район {region_name} не найден.")
                return None
        else:
            print(f"Некорректный ответ API для района {region_name}.")
            return None


# Получаем ID для каждого района из списка
DISTRICT_IDS = {}
for district in MOSCOW_DISTRICTS:
    district_id = get_region_id(district)
    if district_id:
        DISTRICT_IDS[district] = district_id
    time.sleep(0.2)  # Небольшая задержка между запросами, чтобы не превысить лимиты API

In [ ]:
DISTRICT_IDS

{'Академический': '4504209512726625',
 'Алексеевский': '4504209512726568',
 'Алтуфьевский': '4504209512726557',
 'Арбат': '4504209512726530',
 'Аэропорт': '4504209512726541',
 'Бабушкинский': '4504209512726564',
 'Басманный': '4504209512726531',
 'Беговой': '4504209512726542',
 'Бекасово': '4504209520932718',
 'Бескудниковский': '4504209512726543',
 'Бибирево': '4504209512726565',
 'Бирюлёво Восточное': '4504209512726611',
 'Бирюлёво Западное': '4504209512726612',
 'Богородское': '4504209512726574',
 'Братеево': '4504209512726609',
 'Бутырский': '4504209512726567',
 'Вешняки': '4504209512726578',
 'Внуково': '4504209512726642',
 'Войковский': '4504209512726551',
 'Вороново': '4504209520931533',
 'Восточное Дегунино': '4504209512726544',
 'Восточное Измайлово': '4504209512726575',
 'Восточный': '4504209512726586',
 'Выхино-Жулебино': '4504209512726600',
 'Гагаринский': '4504209512726624',
 'Головинский': '4504209512726549',
 'Гольяново': '4504209512726589',
 'Даниловский': '450420951272

In [ ]:
API_KEY = "f50ee13f-107c-47c0-bae5-bf3feb6750bc"

Создаем файл для логирования.

In [ ]:
import json

config = {
    "logging": {
        "enabled": True, #включено/выключено
        "level": "INFO", #уровень логирования
        "log_to_file": True, #сохранение логов в файл
        "log_to_console": True, #вывод логов в консоль
        "log_directory": "logs"
    },
    "api": {
        "key": "f50ee13f-107c-47c0-bae5-bf3feb6750bc",
        "timeout": 10,
        "sleep_between_requests": 0.5
    },
    "data_collection": {
        "page_size": 10,
        "max_pages": 5,
        "sleep_between_districts": 1
    }
}

# Сохраняем в файл config.json
with open('config.json', 'w', encoding='utf-8') as f:
    json.dump(config, f, ensure_ascii=False, indent=4)


Файл config.json успешно создан!


На сайте 2гис при парсинге есть ограничение: разрешено парсить максимум 5 первых страниц и максимум по 10 элементов с каждой из страниц. Парсим всё по-максимому для каждого из 132 районов Москвы.

#### Школы

In [ ]:
import requests
import time
import pandas as pd
import logging
import logging.config
from datetime import datetime
import os
import json

# Загрузка конфигурации из файла
def load_config():
    """Загружает конфигурацию из JSON файла"""
    try:
        with open('config.json', 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print("Файл config.json не найден. Используются настройки по умолчанию.")
        return get_default_config()
    except json.JSONDecodeError as e:
        print(f"Ошибка чтения config.json: {e}. Используются настройки по умолчанию.")
        return get_default_config()

def get_default_config():
    """Возвращает конфигурацию по умолчанию"""
    return {
        "logging": {
            "enabled": True,
            "level": "INFO",
            "log_to_file": True,
            "log_to_console": True,
            "log_directory": "logs"
        },
        "api": {
            "key": "f50ee13f-107c-47c0-bae5-bf3feb6750bc",
            "timeout": 10,
            "sleep_between_requests": 0.5
        },
        "data_collection": {
            "page_size": 10,
            "max_pages": 5,
            "sleep_between_districts": 1
        }
    }

# Загружаем конфигурацию
CONFIG = load_config()


# Настройка логирования на основе конфига
def setup_logging():
    """Настройка системы логирования из конфигурационного файла"""

    if not CONFIG['logging']['enabled']:
        # Выключаем логирование
        logging.getLogger().setLevel(logging.CRITICAL)
        return

    # Создаем папку для логов если её нет
    log_dir = CONFIG['logging']['log_directory']
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # Преобразуем строковый уровень в числовой
    level_name = CONFIG['logging']['level']
    level = getattr(logging, level_name.upper(), logging.INFO)

    # Конфигурация логгера
    logger_config = {
        'version': 1,
        'disable_existing_loggers': False,
        'formatters': {
            'detailed': {
                'format': '%(asctime)s - %(levelname)s - [%(funcName)s] - %(message)s'
            },
            'simple': {
                'format': '%(levelname)s - %(message)s'
            }
        },
        'handlers': {
            'file': {
                'class': 'logging.FileHandler',
                'filename': f"{log_dir}/schools_collection_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log",
                'formatter': 'detailed',
                'encoding': 'utf-8'
            },
            'console': {
                'class': 'logging.StreamHandler',
                'formatter': 'simple',
                'level': 'INFO'
            }
        },
        'root': {
            'level': level,
            'handlers': []
        }
    }

    # Добавляем обработчики в зависимости от настроек
    if CONFIG['logging']['log_to_file']:
        logger_config['root']['handlers'].append('file')
    if CONFIG['logging']['log_to_console']:
        logger_config['root']['handlers'].append('console')

    # Применяем конфигурацию
    logging.config.dictConfig(logger_config)

# Настраиваем логирование
setup_logging()

def get_schools_in_district(district_id, district_name):
    """Функция для получения всех школ в указанном районе"""
    logger = logging.getLogger(__name__)
    all_schools = []
    page = 1
    page_size = CONFIG['data_collection']['page_size']
    max_pages = CONFIG['data_collection']['max_pages']

    logger.info(f"Начало сбора школ для района: {district_name} (ID: {district_id})")

    while page <= max_pages:
        params = {
            'key': CONFIG['api']['key'],
            'q': 'школа',
            'district_id': district_id,
            'type': 'branch',
            'page': page,
            'page_size': page_size,
            'fields': 'items.point,items.name,items.address_name,items.reviews'
        }

        url = "https://catalog.api.2gis.com/3.0/items"

        logger.debug(f"Запрос страницы {page} для района {district_name}")

        try:
            response = requests.get(url, params=params, timeout=CONFIG['api']['timeout'])
            logger.debug(f"Статус ответа: {response.status_code}")

            if response.status_code == 200:
                data = response.json()

                if 'result' in data and 'items' in data['result']:
                    schools = data['result']['items']

                    if schools:
                        logger.info(f"Загружено {len(schools)} школ со страницы {page}")
                        for school in schools:
                            school['district_name'] = district_name
                        all_schools.extend(schools)
                    else:
                        logger.debug(f"Страница {page} пуста")
                else:
                    logger.warning(f"На странице {page} нет данных в ожидаемом формате")

                page += 1

            else:
                logger.error(f"HTTP ошибка: {response.status_code} для страницы {page}")
                page += 1

        except requests.exceptions.Timeout:
            logger.error(f"Таймаут запроса на странице {page}")
            page += 1
        except Exception as e:
            logger.error(f"Ошибка на странице {page}: {e}")
            page += 1

        if page <= max_pages:
            time.sleep(CONFIG['api']['sleep_between_requests'])

    logger.info(f"Завершение сбора для района {district_name}. Найдено школ: {len(all_schools)}")
    return all_schools

def create_dataset(all_schools):
    """Создает структурированный датасет из собранных данных"""
    logger = logging.getLogger(__name__)
    logger.info(f"Создание датасета из {len(all_schools)} школ")

    data = []

    for school in all_schools:
        reviews_data = school.get('reviews', {})
        point_data = school.get('point', {})

        row = {
            'district': school.get('district_name', ''),
            'type': 'школа',
            'name': school.get('name', ''),
            'address': school.get('address_name', ''),
            'latitude': point_data.get('lat', ''),
            'longitude': point_data.get('lon', ''),
            'rating': reviews_data.get('general_rating', 'нет рейтинга'),
            'reviews_count': reviews_data.get('general_review_count', 0),
            'ratings_count': reviews_data.get('general_review_count_with_stars', 0)
        }
        data.append(row)

    logger.info(f"Датасет создан. Количество строк: {len(data)}")
    return pd.DataFrame(data)

# Основной код парсинга
if __name__ == "__main__":
    logger = logging.getLogger(__name__)

    logger.info("=" * 60)
    logger.info("ЗАПУСК СБОРА ДАННЫХ ПО ШКОЛАМ МОСКВЫ")
    logger.info(f"Конфигурация логирования: уровень={CONFIG['logging']['level']}, файл={CONFIG['logging']['log_to_file']}, консоль={CONFIG['logging']['log_to_console']}")
    logger.info(f"Количество районов для обработки: {len(DISTRICT_IDS)}")
    logger.info("=" * 60)

    all_schools_moscow = []
    start_time = time.time()

    # Обрабатываем каждый район
    for i, (district_name, district_id) in enumerate(DISTRICT_IDS.items(), 1):
        logger.info(f"[{i}/{len(DISTRICT_IDS)}] Обработка района: {district_name}")

        try:
            schools_in_district = get_schools_in_district(district_id, district_name)
            all_schools_moscow.extend(schools_in_district)
            logger.info(f"Район '{district_name}' завершен. Школ: {len(schools_in_district)}")

        except Exception as e:
            logger.error(f"Ошибка в районе {district_name}: {e}")
            continue

        if i < len(DISTRICT_IDS):
            time.sleep(CONFIG['data_collection']['sleep_between_districts'])

    # Финализация
    end_time = time.time()
    execution_time = end_time - start_time

    logger.info("=" * 60)
    logger.info(f"СБОР ДАННЫХ ЗАВЕРШЕН")
    logger.info(f"Время выполнения: {execution_time:.2f} секунд")
    logger.info(f"Всего собрано школ: {len(all_schools_moscow)}")
    logger.info("=" * 60)

    # Создаем датасет
    if all_schools_moscow:
        df1 = create_dataset(all_schools_moscow)
        filename = 'школы_москвы_по_районам.csv'
        df1.to_csv(filename, index=False, encoding='utf-8')
        logger.info(f"Данные сохранены в {filename}")

        logger.info(f"СТАТИСТИКА: Всего школ: {len(df1)}")
        logger.info(f"СТАТИСТИКА: Охвачено районов: {df1['district'].nunique()}")

    else:
        logger.error("Не удалось собрать данные о парках")
        print("Не удалось собрать данные о парках")

INFO - ============================================================
INFO - ЗАПУСК СБОРА ДАННЫХ ПО ШКОЛАМ МОСКВЫ
INFO - Конфигурация логирования: уровень=INFO, файл=True, консоль=True
INFO - Количество районов для обработки: 132
INFO - ============================================================
INFO - [1/132] Обработка района: Академический
INFO - Начало сбора школ для района: Академический (ID: 4504209512726625)
INFO - Загружено 10 школ со страницы 1
INFO - Загружено 9 школ со страницы 2
WARNING - На странице 3 нет данных в ожидаемом формате
WARNING - На странице 4 нет данных в ожидаемом формате
WARNING - На странице 5 нет данных в ожидаемом формате
INFO - Завершение сбора для района Академический. Найдено школ: 19
INFO - Район 'Академический' завершен. Школ: 19
INFO - [2/132] Обработка района: Алексеевский
INFO - Начало сбора школ для района: Алексеевский (ID: 4504209512726568)
INFO - Загружено 10 школ со страницы 1
INFO - Загружено 6 школ со страницы 2
WARNING - На странице 3 нет да

NameError: name 'df' is not defined

In [ ]:
df1

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Академический,школа,Академическая школа №1534,"улица Гримау, 11а ст3",55.687755,37.582081,4.4,6,24
1,Академический,школа,"Лит, школа №1533","Профсоюзная улица, 24 к4",55.680605,37.562167,5,1,6
2,Академический,школа,"Академическая школа №1534, главный корпус","улица Кедрова, 11",55.684737,37.574167,4.1,4,21
3,Академический,школа,Школа №199,"улица Дмитрия Ульянова, 15 к1",55.690083,37.571609,5,1,6
4,Академический,школа,Школа №45,"улица Гримау, 8",55.691589,37.580152,4.5,2,10
...,...,...,...,...,...,...,...,...,...
2471,Ясенево,школа,"Школа №1103, школа им. Героя РФ А.В. Соломатина","Литовский бульвар, 42 к2",55.608020,37.541734,4.9,1,4
2472,Ясенево,школа,Школа №2103,"Голубинская улица, 13 к2",55.603158,37.517547,нет рейтинга,0,2
2473,Ясенево,школа,"Школа им. Н.М. Карамзина, Остафьевский корпус №9","проезд Карамзина, 9 к2",55.591577,37.540696,нет рейтинга,0,0
2474,Ясенево,школа,"London Express, школа","Соловьиный проезд, 1",55.619451,37.543880,5,3,5


In [ ]:
df1['район'].nunique()

132

Во всех районах Москвы есть школы.

#### Парки

In [ ]:
import requests
import time
import pandas as pd
import logging
import logging.config
from datetime import datetime
import os
import json

# Загрузка конфигурации из файла
def load_config():
    """Загружает конфигурацию из JSON файла"""
    try:
        with open('config.json', 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print("Файл config.json не найден. Используются настройки по умолчанию.")
        return get_default_config()
    except json.JSONDecodeError as e:
        print(f"Ошибка чтения config.json: {e}. Используются настройки по умолчанию.")
        return get_default_config()

def get_default_config():
    """Возвращает конфигурацию по умолчанию"""
    return {
        "logging": {
            "enabled": True,
            "level": "INFO",
            "log_to_file": True,
            "log_to_console": True,
            "log_directory": "logs"
        },
        "api": {
            "key": "f50ee13f-107c-47c0-bae5-bf3feb6750bc",
            "timeout": 10,
            "sleep_between_requests": 0.5
        },
        "data_collection": {
            "page_size": 10,
            "max_pages": 5,
            "sleep_between_districts": 1
        }
    }

# Загружаем конфигурацию
CONFIG = load_config()


# Настройка логирования на основе конфига
def setup_logging():
    """Настройка системы логирования из конфигурационного файла"""

    if not CONFIG['logging']['enabled']:
        # Выключаем логирование
        logging.getLogger().setLevel(logging.CRITICAL)
        return

    # Создаем папку для логов если её нет
    log_dir = CONFIG['logging']['log_directory']
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # Преобразуем строковый уровень в числовой
    level_name = CONFIG['logging']['level']
    level = getattr(logging, level_name.upper(), logging.INFO)

    # Конфигурация логгера
    logger_config = {
        'version': 1,
        'disable_existing_loggers': False,
        'formatters': {
            'detailed': {
                'format': '%(asctime)s - %(levelname)s - [%(funcName)s] - %(message)s'
            },
            'simple': {
                'format': '%(levelname)s - %(message)s'
            }
        },
        'handlers': {
            'file': {
                'class': 'logging.FileHandler',
                'filename': f"{log_dir}/parks_collection_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log",
                'formatter': 'detailed',
                'encoding': 'utf-8'
            },
            'console': {
                'class': 'logging.StreamHandler',
                'formatter': 'simple',
                'level': 'INFO'
            }
        },
        'root': {
            'level': level,
            'handlers': []
        }
    }

    # Добавляем обработчики в зависимости от настроек
    if CONFIG['logging']['log_to_file']:
        logger_config['root']['handlers'].append('file')
    if CONFIG['logging']['log_to_console']:
        logger_config['root']['handlers'].append('console')

    # Применяем конфигурацию
    logging.config.dictConfig(logger_config)

# Настраиваем логирование
setup_logging()

def get_parks_in_district(district_id, district_name):
    """Функция для получения всех парков в указанном районе"""

    logger = logging.getLogger(__name__)
    all_parks = []
    page = 1
    page_size = CONFIG['data_collection']['page_size']
    max_pages = CONFIG['data_collection']['max_pages']

    logger.info(f"Начало сбора парков для района: {district_name} (ID: {district_id})")

    while page <= max_pages:
        params = {
            'key': CONFIG['api']['key'],
            'q': 'парк',
            'district_id': district_id,
            'type': 'branch',
            'page': page,
            'page_size': page_size,
            'fields': 'items.point,items.name,items.address_name,items.reviews'
        }

        url = "https://catalog.api.2gis.com/3.0/items"

        logger.debug(f"Запрос страницы {page} для района {district_name}")

        try:
            response = requests.get(url, params=params, timeout=CONFIG['api']['timeout'])
            logger.debug(f"Статус ответа: {response.status_code}")

            if response.status_code == 200:
                data = response.json()

                if 'result' in data and 'items' in data['result']:
                    parks = data['result']['items']

                    if parks:
                        logger.info(f"Загружено {len(parks)} парков со страницы {page}")
                        for park in parks:
                            park['district_name'] = district_name
                        all_parks.extend(parks)
                    else:
                        logger.debug(f"Страница {page} пуста")
                else:
                    logger.warning(f"На странице {page} нет данных в ожидаемом формате")

                page += 1

            else:
                logger.error(f"HTTP ошибка: {response.status_code} для страницы {page}")
                page += 1

        except requests.exceptions.Timeout:
            logger.error(f"Таймаут запроса на странице {page}")
            page += 1
        except Exception as e:
            logger.error(f"Ошибка на странице {page}: {e}")
            page += 1

        if page <= max_pages:
            time.sleep(CONFIG['api']['sleep_between_requests'])

    logger.info(f"Завершение сбора для района {district_name}. Найдено парков: {len(all_parks)}")
    return all_parks

def create_parks_dataset(all_parks):
    """Создает структурированный датасет из собранных данных о парках"""

    logger = logging.getLogger(__name__)
    logger.info(f"Создание датасета из {len(all_parks)} парков")

    data = []

    for park in all_parks:
        reviews_data = park.get('reviews', {})
        point_data = park.get('point', {})

        row = {
            'district': park.get('district_name', ''),
            'type': 'парк',
            'name': park.get('name', ''),
            'address': park.get('address_name', ''),
            'latitude': point_data.get('lat', ''),
            'longitude': point_data.get('lon', ''),
            'rating': reviews_data.get('general_rating', 'нет рейтинга'),
            'reviews_count': reviews_data.get('general_review_count', 0),
            'ratings_count': reviews_data.get('general_review_count_with_stars', 0)
        }
        data.append(row)

    logger.info(f"Датасет парков создан. Количество строк: {len(data)}")
    return pd.DataFrame(data)

# Основной код
if __name__ == "__main__":
    logger = logging.getLogger(__name__)

    logger.info("=" * 60)
    logger.info("ЗАПУСК СБОРА ДАННЫХ ПО ПАРКАМ МОСКВЫ")
    logger.info(f"Конфигурация логирования: уровень={CONFIG['logging']['level']}, файл={CONFIG['logging']['log_to_file']}, консоль={CONFIG['logging']['log_to_console']}")
    logger.info(f"Количество районов для обработки: {len(DISTRICT_IDS)}")
    logger.info("=" * 60)

    all_parks_moscow = []
    start_time = time.time()

    # Обрабатываем каждый район
    for i, (district_name, district_id) in enumerate(DISTRICT_IDS.items(), 1):
        logger.info(f"[{i}/{len(DISTRICT_IDS)}] Обработка района: {district_name}")

        try:
            parks_in_district = get_parks_in_district(district_id, district_name)
            all_parks_moscow.extend(parks_in_district)
            logger.info(f"Район '{district_name}' завершен. Парков: {len(parks_in_district)}")

        except Exception as e:
            logger.error(f"Ошибка в районе {district_name}: {e}")
            continue

        if i < len(DISTRICT_IDS):
            time.sleep(CONFIG['data_collection']['sleep_between_districts'])

    # Финализация
    end_time = time.time()
    execution_time = end_time - start_time

    logger.info("=" * 60)
    logger.info(f"СБОР ДАННЫХ ЗАВЕРШЕН")
    logger.info(f"Время выполнения: {execution_time:.2f} секунд")
    logger.info(f"Всего собрано парков: {len(all_parks_moscow)}")
    logger.info("=" * 60)

    # Создаем датасет df2
    if all_parks_moscow:
        df2 = create_parks_dataset(all_parks_moscow)
        filename = 'парки_москвы_по_районам.csv'
        df2.to_csv(filename, index=False, encoding='utf-8')
        logger.info(f"Данные сохранены в {filename}")

        logger.info(f"СТАТИСТИКА: Всего парков: {len(df2)}")
        logger.info(f"СТАТИСТИКА: Охвачено районов: {df2['district'].nunique()}")

    else:
        logger.error("Не удалось собрать данные о парках")
        print("Не удалось собрать данные о парках")

Парсинг прервался из-за ограничения в запросах с сайта.

Соберем таблицу с загруженными данными.

In [ ]:
df2_ = []

for park in all_parks_moscow:
    reviews_data = park.get('reviews', {})
    point_data = park.get('point', {})

    row = {
        'район': park.get('district_name', ''),
        'тип': 'парк',
        'название': park.get('name', ''),
        'адрес': park.get('address_name', ''),
        'широта': point_data.get('lat', ''),
        'долгота': point_data.get('lon', ''),
        'рейтинг': reviews_data.get('general_rating', 'нет рейтинга'),
        'количество_отзывов': reviews_data.get('general_review_count', 0),
        'количество_оценок': reviews_data.get('general_review_count_with_stars', 0)
    }
    df2_.append(row)
df2_ = pd.DataFrame(df2_)

In [ ]:
df2_

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Академический,парк,"Фестиваль-парк, компания по продаже, прокату и...","Новочерёмушкинская улица, 25",55.680951,37.579157,4,1,1
1,Алтуфьевский,парк,"Бибирево, 3-й автобусный парк","Бибиревская улица, 2 ст1",55.877055,37.588744,3.8,7,17
2,Арбат,парк,"Cosmos Selection Moscow Arbat, гостиница","улица Новый Арбат, 2",55.753181,37.599468,4.9,36,73
3,Аэропорт,парк,Петровский парк,,55.793866,37.557141,4.7,99,669
4,Аэропорт,парк,"Авиапарк, офисный центр","Ленинградский проспект, 68 ст2",55.804713,37.523052,4.6,2,15
...,...,...,...,...,...,...,...,...,...
278,Кунцево,парк,"Мякинино парк, жилой комплекс",,55.804413,37.347617,5,2,3
279,Кунцево,парк,"Мякинино парк, жилой комплекс",,55.803318,37.346029,5,3,4
280,Левобережный,парк,Парк Дружбы,,55.852773,37.479729,4.9,217,975
281,Левобережный,парк,Парк Северного речного вокзала,,55.850933,37.469567,4.9,253,1248


In [ ]:
df2_.to_csv('половинаПАРКОВ.csv', index=False, encoding='utf-8')

Прервался на районе Левобережный, поэтому далее с новым ключом продолжим парсить именно с этого района.

In [ ]:
DISTRICT_IDS1 = {'Левобережный': '4504209512726554',
 'Лефортово': '4504209512726590',
 'Лианозово': '4504209512726559',
 'Ломоносовский': '4504209512726623',
 'Лосиноостровский': '4504209512726560',
 'Люблино': '70030077079195278',
 'Марфино': '4504209512726570',
 'Марьина Роща': '4504209512726571',
 'Марьино': '4504209512726596',
 'Матушкино': '4504209512726657',
 'Метрогородок': '4504209512726588',
 'Мещанский': '4504209512726535',
 'Митино': '4504209512726645',
 'Можайский': '4504209512726633',
 'Молжаниновский': '4504209512726654',
 'Москворечье-Сабурово': '4504209512726607',
 'Нагатино-Садовники': '4504209512726604',
 'Нагатинский Затон': '4504209512726603',
 'Нагорный': '4504209512726606',
 'Некрасовка': '4504209512726599',
 'Нижегородский': '4504209512726591',
 'Новогиреево': '4504209512726580',
 'Новокосино': '4504209512726576',
 'Ново-Переделкино': '4504209512726641',
 'Обручевский': '4504209512726622',
 'Орехово-Борисово Северное': '4504209512726616',
 'Орехово-Борисово Южное': '4504209512726610',
 'Останкинский': '4504209512726572',
 'Отрадное': '4504209512726569',
 'Очаково-Матвеевское': '4504209512726632',
 'Перово': '4504209512726581',
 'Печатники': '4504209512726593',
 'Покровское-Стрешнево': '4504209512726650',
 'Преображенское': '4504209512726584',
 'Пресненский': '4504209512726539',
 'Проспект Вернадского': '4504209512726631',
 'Раменки': '4504209512726639',
 'Ростокино': '4504209512726558',
 'Рязанский': '4504209512726595',
 'Савёлки': '4504209512726658',
 'Савёловский': '4504209512726548',
 'Свиблово': '4504209512726563',
 'Северное Бутово': '4504209512726618',
 'Северное Измайлово': '4504209512726583',
 'Северное Медведково': '4504209512726566',
 'Северное Тушино': '4504209512726646',
 'Северный': '4504209512726573',
 'Силино': '4504209512726656',
 'Сокол': '4504209512726552',
 'Соколиная Гора': '4504209512726585',
 'Сокольники': '4504209512726582',
 'Солнцево': '4504209512726640',
 'Старое Крюково': '4504209512726655',
 'Строгино': '4504209512726644',
 'Таганский': '4504209512726538',
 'Тверской': '4504209512726536',
 'Текстильщики': '4504209512726594',
 'Тёплый Стан': '4504209512726621',
 'Тимирязевский': '4504209512726547',
 'Троицк': '4504209520933741',
 'Тропарёво-Никулино': '4504209512726630',
 'Филёвский Парк': '4504209512726636',
 'Фили-Давыдково': '4504209512726638',
 'Филимонковский': '4504209520929329',
 'Хамовники': '4504209512726533',
 'Ховрино': '4504209512726555',
 'Хорошёво-Мнёвники': '4504209512726647',
 'Хорошёвский': '4504209512726553',
 'Царицыно': '4504209512726608',
 'Черёмушки': '4504209512726627',
 'Чертаново Северное': '4504209512726615',
 'Чертаново Центральное': '4504209512726614',
 'Чертаново Южное': '4504209512726659',
 'Щербинка': '4504209520928047',
 'Щукино': '4504209512726648',
 'Южное Бутово': '4504209512726619',
 'Южное Медведково': '4504209512726561',
 'Южное Тушино': '4504209512726649',
 'Южнопортовый': '70030076266239145',
 'Якиманка': '4504209512726534',
 'Ярославский': '4504209512726562',
 'Ясенево': '4504209512726620'}

In [ ]:
API_KEY = "2e3b8ee1-dca8-47ab-99a0-bb8cfce53698"

In [ ]:
import requests
import time
import pandas as pd
import logging
import logging.config
from datetime import datetime
import os
import json

# Загрузка конфигурации из файла
def load_config():
    """Загружает конфигурацию из JSON файла"""
    try:
        with open('config.json', 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print("Файл config.json не найден. Используются настройки по умолчанию.")
        return get_default_config()
    except json.JSONDecodeError as e:
        print(f"Ошибка чтения config.json: {e}. Используются настройки по умолчанию.")
        return get_default_config()

def get_default_config():
    """Возвращает конфигурацию по умолчанию"""
    return {
        "logging": {
            "enabled": True,
            "level": "INFO",
            "log_to_file": True,
            "log_to_console": True,
            "log_directory": "logs"
        },
        "api": {
            "key": "f50ee13f-107c-47c0-bae5-bf3feb6750bc",
            "timeout": 10,
            "sleep_between_requests": 0.5
        },
        "data_collection": {
            "page_size": 10,
            "max_pages": 5,
            "sleep_between_districts": 1
        }
    }

# Загружаем конфигурацию
CONFIG = load_config()


# Настройка логирования на основе конфига
def setup_logging():
    """Настройка системы логирования из конфигурационного файла"""

    if not CONFIG['logging']['enabled']:
        # Выключаем логирование
        logging.getLogger().setLevel(logging.CRITICAL)
        return

    # Создаем папку для логов если её нет
    log_dir = CONFIG['logging']['log_directory']
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # Преобразуем строковый уровень в числовой
    level_name = CONFIG['logging']['level']
    level = getattr(logging, level_name.upper(), logging.INFO)

    # Конфигурация логгера
    logger_config = {
        'version': 1,
        'disable_existing_loggers': False,
        'formatters': {
            'detailed': {
                'format': '%(asctime)s - %(levelname)s - [%(funcName)s] - %(message)s'
            },
            'simple': {
                'format': '%(levelname)s - %(message)s'
            }
        },
        'handlers': {
            'file': {
                'class': 'logging.FileHandler',
                'filename': f"{log_dir}/parks_collection_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log",
                'formatter': 'detailed',
                'encoding': 'utf-8'
            },
            'console': {
                'class': 'logging.StreamHandler',
                'formatter': 'simple',
                'level': 'INFO'
            }
        },
        'root': {
            'level': level,
            'handlers': []
        }
    }

    # Добавляем обработчики в зависимости от настроек
    if CONFIG['logging']['log_to_file']:
        logger_config['root']['handlers'].append('file')
    if CONFIG['logging']['log_to_console']:
        logger_config['root']['handlers'].append('console')

    # Применяем конфигурацию
    logging.config.dictConfig(logger_config)

# Настраиваем логирование
setup_logging()

def get_parks_in_district(district_id, district_name):
    """Функция для получения всех парков в указанном районе"""

    logger = logging.getLogger(__name__)
    all_parks = []
    page = 1
    page_size = CONFIG['data_collection']['page_size']
    max_pages = CONFIG['data_collection']['max_pages']

    logger.info(f"Начало сбора парков для района: {district_name} (ID: {district_id})")

    while page <= max_pages:
        params = {
            'key': CONFIG['api']['key'],
            'q': 'парк',
            'district_id': district_id,
            'type': 'branch',
            'page': page,
            'page_size': page_size,
            'fields': 'items.point,items.name,items.address_name,items.reviews'
        }

        url = "https://catalog.api.2gis.com/3.0/items"

        logger.debug(f"Запрос страницы {page} для района {district_name}")

        try:
            response = requests.get(url, params=params, timeout=CONFIG['api']['timeout'])
            logger.debug(f"Статус ответа: {response.status_code}")

            if response.status_code == 200:
                data = response.json()

                if 'result' in data and 'items' in data['result']:
                    parks = data['result']['items']

                    if parks:
                        logger.info(f"Загружено {len(parks)} парков со страницы {page}")
                        for park in parks:
                            park['district_name'] = district_name
                        all_parks.extend(parks)
                    else:
                        logger.debug(f"Страница {page} пуста")
                else:
                    logger.warning(f"На странице {page} нет данных в ожидаемом формате")

                page += 1

            else:
                logger.error(f"HTTP ошибка: {response.status_code} для страницы {page}")
                page += 1

        except requests.exceptions.Timeout:
            logger.error(f"Таймаут запроса на странице {page}")
            page += 1
        except Exception as e:
            logger.error(f"Ошибка на странице {page}: {e}")
            page += 1

        if page <= max_pages:
            time.sleep(CONFIG['api']['sleep_between_requests'])

    logger.info(f"Завершение сбора для района {district_name}. Найдено парков: {len(all_parks)}")
    return all_parks

def create_parks_dataset(all_parks):
    """Создает структурированный датасет из собранных данных о парках"""

    logger = logging.getLogger(__name__)
    logger.info(f"Создание датасета из {len(all_parks)} парков")

    data = []

    for park in all_parks:
        reviews_data = park.get('reviews', {})
        point_data = park.get('point', {})

        row = {
            'district': park.get('district_name', ''),
            'type': 'парк',
            'name': park.get('name', ''),
            'address': park.get('address_name', ''),
            'latitude': point_data.get('lat', ''),
            'longitude': point_data.get('lon', ''),
            'rating': reviews_data.get('general_rating', 'нет рейтинга'),
            'reviews_count': reviews_data.get('general_review_count', 0),
            'ratings_count': reviews_data.get('general_review_count_with_stars', 0)
        }
        data.append(row)

    logger.info(f"Датасет парков создан. Количество строк: {len(data)}")
    return pd.DataFrame(data)

# Основной код
if __name__ == "__main__":
    logger = logging.getLogger(__name__)

    logger.info("=" * 60)
    logger.info("ЗАПУСК СБОРА ДАННЫХ ПО ПАРКАМ МОСКВЫ")
    logger.info(f"Конфигурация логирования: уровень={CONFIG['logging']['level']}, файл={CONFIG['logging']['log_to_file']}, консоль={CONFIG['logging']['log_to_console']}")
    logger.info(f"Количество районов для обработки: {len(DISTRICT_IDS1)}")
    logger.info("=" * 60)

    all_parks_moscow = []
    start_time = time.time()

    # Обрабатываем каждый район
    for i, (district_name, district_id) in enumerate(DISTRICT_IDS1.items(), 1):
        logger.info(f"[{i}/{len(DISTRICT_IDS1)}] Обработка района: {district_name}")

        try:
            parks_in_district = get_parks_in_district(district_id, district_name)
            all_parks_moscow.extend(parks_in_district)
            logger.info(f"Район '{district_name}' завершен. Парков: {len(parks_in_district)}")

        except Exception as e:
            logger.error(f"Ошибка в районе {district_name}: {e}")
            continue

        if i < len(DISTRICT_IDS1):
            time.sleep(CONFIG['data_collection']['sleep_between_districts'])

    # Финализация
    end_time = time.time()
    execution_time = end_time - start_time

    logger.info("=" * 60)
    logger.info(f"СБОР ДАННЫХ ЗАВЕРШЕН")
    logger.info(f"Время выполнения: {execution_time:.2f} секунд")
    logger.info(f"Всего собрано парков: {len(all_parks_moscow)}")
    logger.info("=" * 60)

    # Создаем датасет df2
    if all_parks_moscow:
        df2 = create_parks_dataset(all_parks_moscow)
        filename = 'парки_москвы_по_районам.csv'
        df2.to_csv(filename, index=False, encoding='utf-8')
        logger.info(f"Данные сохранены в {filename}")

        logger.info(f"СТАТИСТИКА: Всего парков: {len(df2)}")
        logger.info(f"СТАТИСТИКА: Охвачено районов: {df2['district'].nunique()}")

    else:
        logger.error("Не удалось собрать данные о парках")
        print("Не удалось собрать данные о парках")

In [ ]:
df2

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Левобережный,парк,Парк Дружбы,,55.852773,37.479729,4.9,217,975
1,Левобережный,парк,Парк Северного речного вокзала,,55.850933,37.469567,4.9,253,1248
2,Левобережный,парк,"Фестиваль парк, жилой комплекс","Фестивальная улица, 15 к6",55.858424,37.476206,нет рейтинга,0,0
3,Лефортово,парк,Лефортовский парк,,55.765181,37.690675,4.8,145,411
4,Лианозово,парк,"Лианозовский, парк культуры и отдыха",,55.900408,37.571020,4.8,64,252
...,...,...,...,...,...,...,...,...,...
444,Ясенево,парк,"Принц парк отель, гостиничный комплекс","Новоясеневский проспект, 1Б к1",55.617523,37.511901,4.2,756,1814
445,Ясенево,парк,"Prime Hall, банкетный зал","Новоясеневский проспект, 1Б к1",55.617516,37.511569,5,1,4
446,Ясенево,парк,"Joki Joya, парк развлечений","Новоясеневский проспект, 1",55.619968,37.509122,4.7,80,97
447,Ясенево,парк,"Scandy Park, детский развлекательный центр","Новоясеневский проспект, 7",55.608167,37.532359,4.6,27,29


In [ ]:
df2 = pd.concat([df2_, df2], axis=0)

Удалим дубликаты, которые могут быть из-за повторного парсинга данных по району Левобережный.

In [ ]:
df2 = df2.drop_duplicates()

In [ ]:
df2 = df2.reset_index()
del df2['index']
df2

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Академический,парк,"Фестиваль-парк, компания по продаже, прокату и...","Новочерёмушкинская улица, 25",55.680951,37.579157,4,1,1
1,Алтуфьевский,парк,"Бибирево, 3-й автобусный парк","Бибиревская улица, 2 ст1",55.877055,37.588744,3.8,7,17
2,Арбат,парк,"Cosmos Selection Moscow Arbat, гостиница","улица Новый Арбат, 2",55.753181,37.599468,4.9,36,73
3,Аэропорт,парк,Петровский парк,,55.793866,37.557141,4.7,99,669
4,Аэропорт,парк,"Авиапарк, офисный центр","Ленинградский проспект, 68 ст2",55.804713,37.523052,4.6,2,15
...,...,...,...,...,...,...,...,...,...
724,Ясенево,парк,"Принц парк отель, гостиничный комплекс","Новоясеневский проспект, 1Б к1",55.617523,37.511901,4.2,756,1814
725,Ясенево,парк,"Prime Hall, банкетный зал","Новоясеневский проспект, 1Б к1",55.617516,37.511569,5,1,4
726,Ясенево,парк,"Joki Joya, парк развлечений","Новоясеневский проспект, 1",55.619968,37.509122,4.7,80,97
727,Ясенево,парк,"Scandy Park, детский развлекательный центр","Новоясеневский проспект, 7",55.608167,37.532359,4.6,27,29


In [ ]:
len(df2['район'].value_counts())

114

Не во всех районах Москвы есть парки.


In [ ]:
df2.to_csv('парки_москвы_по_районам.csv', index=False, encoding='utf-8')

#### Поликлиники

In [ ]:
import requests
import time
import pandas as pd
import logging
import logging.config
from datetime import datetime
import os
import json

# Загрузка конфигурации из файла
def load_config():
    """Загружает конфигурацию из JSON файла"""
    try:
        with open('config.json', 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print("Файл config.json не найден. Используются настройки по умолчанию.")
        return get_default_config()
    except json.JSONDecodeError as e:
        print(f"Ошибка чтения config.json: {e}. Используются настройки по умолчанию.")
        return get_default_config()

def get_default_config():
    """Возвращает конфигурацию по умолчанию"""
    return {
        "logging": {
            "enabled": True,
            "level": "INFO",
            "log_to_file": True,
            "log_to_console": True,
            "log_directory": "logs"
        },
        "api": {
            "key": "2e3b8ee1-dca8-47ab-99a0-bb8cfce53698",
            "timeout": 10,
            "sleep_between_requests": 0.5
        },
        "data_collection": {
            "page_size": 10,
            "max_pages": 5,
            "sleep_between_districts": 1
        }
    }

# Загружаем конфигурацию
CONFIG = load_config()


# Настройка логирования на основе конфига
def setup_logging():
    """Настройка системы логирования из конфигурационного файла"""

    if not CONFIG['logging']['enabled']:
        # Выключаем логирование
        logging.getLogger().setLevel(logging.CRITICAL)
        return

    # Создаем папку для логов если её нет
    log_dir = CONFIG['logging']['log_directory']
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # Преобразуем строковый уровень в числовой
    level_name = CONFIG['logging']['level']
    level = getattr(logging, level_name.upper(), logging.INFO)

    # Конфигурация логгера
    logger_config = {
        'version': 1,
        'disable_existing_loggers': False,
        'formatters': {
            'detailed': {
                'format': '%(asctime)s - %(levelname)s - [%(funcName)s] - %(message)s'
            },
            'simple': {
                'format': '%(levelname)s - %(message)s'
            }
        },
        'handlers': {
            'file': {
                'class': 'logging.FileHandler',
                'filename': f"{log_dir}/clinics_collection_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log",  # Изменили имя файла
                'formatter': 'detailed',
                'encoding': 'utf-8'
            },
            'console': {
                'class': 'logging.StreamHandler',
                'formatter': 'simple',
                'level': 'INFO'
            }
        },
        'root': {
            'level': level,
            'handlers': []
        }
    }

    # Добавляем обработчики в зависимости от настроек
    if CONFIG['logging']['log_to_file']:
        logger_config['root']['handlers'].append('file')
    if CONFIG['logging']['log_to_console']:
        logger_config['root']['handlers'].append('console')

    # Применяем конфигурацию
    logging.config.dictConfig(logger_config)

# Настраиваем логирование
setup_logging()

def get_clinics_in_district(district_id, district_name):
    """
    Функция для получения всех поликлиник в указанном районе
    """
    logger = logging.getLogger(__name__)
    all_clinics = []
    page = 1
    page_size = CONFIG['data_collection']['page_size']
    max_pages = CONFIG['data_collection']['max_pages']

    logger.info(f"Начало сбора поликлиник для района: {district_name} (ID: {district_id})")

    while page <= max_pages:
        params = {
            'key': CONFIG['api']['key'],
            'q': 'поликлиника',
            'district_id': district_id,
            'type': 'branch',
            'page': page,
            'page_size': page_size,
            'fields': 'items.point,items.name,items.address_name,items.reviews'
        }

        url = "https://catalog.api.2gis.com/3.0/items"

        logger.debug(f"Запрос страницы {page} для района {district_name}")

        try:
            response = requests.get(url, params=params, timeout=CONFIG['api']['timeout'])
            logger.debug(f"Статус ответа: {response.status_code}")

            if response.status_code == 200:
                data = response.json()

                if 'result' in data and 'items' in data['result']:
                    clinics = data['result']['items']

                    if clinics:
                        logger.info(f"Загружено {len(clinics)} поликлиник со страницы {page}")
                        for clinic in clinics:
                            clinic['district_name'] = district_name
                        all_clinics.extend(clinics)
                    else:
                        logger.debug(f"Страница {page} пуста")
                else:
                    logger.warning(f"На странице {page} нет данных в ожидаемом формате")

                page += 1

            else:
                logger.error(f"HTTP ошибка: {response.status_code} для страницы {page}")
                page += 1

        except requests.exceptions.Timeout:
            logger.error(f"Таймаут запроса на странице {page}")
            page += 1
        except Exception as e:
            logger.error(f"Ошибка на странице {page}: {e}")
            page += 1

        if page <= max_pages:
            time.sleep(CONFIG['api']['sleep_between_requests'])

    logger.info(f"Завершение сбора для района {district_name}. Найдено поликлиник: {len(all_clinics)}")
    return all_clinics

def create_clinics_dataset(all_clinics):
    """
    Создает структурированный датасет из собранных данных о поликлиниках
    """
    logger = logging.getLogger(__name__)
    logger.info(f"Создание датасета из {len(all_clinics)} поликлиник")

    data = []

    for clinic in all_clinics:
        reviews_data = clinic.get('reviews', {})
        point_data = clinic.get('point', {})

        row = {
            'district': clinic.get('district_name', ''),
            'type': 'поликлиника',
            'name': clinic.get('name', ''),
            'address': clinic.get('address_name', ''),
            'latitude': point_data.get('lat', ''),
            'longitude': point_data.get('lon', ''),
            'rating': reviews_data.get('general_rating', 'нет рейтинга'),
            'reviews_count': reviews_data.get('general_review_count', 0),
            'ratings_count': reviews_data.get('general_review_count_with_stars', 0)
        }
        data.append(row)

    logger.info(f"Датасет поликлиник создан. Количество строк: {len(data)}")
    return pd.DataFrame(data)

# Основной код
if __name__ == "__main__":
    logger = logging.getLogger(__name__)

    logger.info("=" * 60)
    logger.info("ЗАПУСК СБОРА ДАННЫХ ПО ПОЛИКЛИНИКАМ МОСКВЫ")
    logger.info(f"Конфигурация логирования: уровень={CONFIG['logging']['level']}, файл={CONFIG['logging']['log_to_file']}, консоль={CONFIG['logging']['log_to_console']}")
    logger.info(f"Количество районов для обработки: {len(DISTRICT_IDS)}")
    logger.info("=" * 60)

    all_clinics_moscow = []
    start_time = time.time()

    # Обрабатываем каждый район
    for i, (district_name, district_id) in enumerate(DISTRICT_IDS.items(), 1):
        logger.info(f"[{i}/{len(DISTRICT_IDS)}] Обработка района: {district_name}")

        try:
            clinics_in_district = get_clinics_in_district(district_id, district_name)
            all_clinics_moscow.extend(clinics_in_district)
            logger.info(f"Район '{district_name}' завершен. Поликлиник: {len(clinics_in_district)}")

        except Exception as e:
            logger.error(f"Ошибка в районе {district_name}: {e}")
            continue

        if i < len(DISTRICT_IDS):
            time.sleep(CONFIG['data_collection']['sleep_between_districts'])

    # Финализация
    end_time = time.time()
    execution_time = end_time - start_time

    logger.info("=" * 60)
    logger.info(f"СБОР ДАННЫХ ЗАВЕРШЕН")
    logger.info(f"Время выполнения: {execution_time:.2f} секунд")
    logger.info(f"Всего собрано поликлиник: {len(all_clinics_moscow)}")
    logger.info("=" * 60)

    # Создаем датасет
    if all_clinics_moscow:
        df3 = create_clinics_dataset(all_clinics_moscow)
        filename = 'поликлиники_москвы_по_районам.csv'
        df3.to_csv(filename, index=False, encoding='utf-8')
        logger.info(f"Данные сохранены в {filename}")

        logger.info(f"СТАТИСТИКА: Всего поликлиник: {len(df3)}")
        logger.info(f"СТАТИСТИКА: Охвачено районов: {df3['district'].nunique()}")

    else:
        logger.error("Не удалось собрать данные о поликлиниках")
        print("Не удалось собрать данные о поликлиниках")

In [ ]:
df3

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Академический,поликлиника,"Поликлиника №22, головное учреждение, отделени...","улица Вавилова, 61а",55.688727,37.559926,5,3,7
1,Академический,поликлиника,"Поликлиника №67, филиал №7","Большая Черёмушкинская улица, 6а",55.687604,37.594333,1.8,5,9
2,Академический,поликлиника,"Поликлиника №22 г. Москвы, гериатрическое отде...","улица Дмитрия Ульянова, 25",55.686235,37.579795,1.3,25,27
3,Академический,поликлиника,Городская поликлиника №22,"улица Вавилова, 61а",55.688860,37.559847,нет рейтинга,0,0
4,Академический,поликлиника,"Детская поликлиника №69, филиал №3","улица Винокурова, 14",55.686545,37.590848,3.1,6,9
...,...,...,...,...,...,...,...,...,...
1060,Ясенево,поликлиника,Детская городская поликлиника №42,"Ясногорская улица, 3а",55.610793,37.539573,4,1,1
1061,Ясенево,поликлиника,"Детская городская поликлиника №42, педиатричес...","Ясногорская улица, 3а",55.610751,37.539543,нет рейтинга,0,0
1062,Ясенево,поликлиника,"Детская городская поликлиника №42, отделение п...","Ясногорская улица, 3а",55.610751,37.539543,нет рейтинга,0,0
1063,Ясенево,поликлиника,"Детская городская поликлиника №42, отделение п...","Голубинская улица, 23 к2",55.599374,37.532350,нет рейтинга,0,0


In [ ]:
len(df3['район'].value_counts())

131

Не во всех районах Москвы есть поликлиники, а именно, только в одном нет.

#### Детские сады

Берем новый ключ, чтобы не прервалось.

In [ ]:
API_KEY = "cdbda41e-e8e4-4e8d-9886-f12c075adc64"

In [ ]:
import requests
import time
import pandas as pd
import logging
import logging.config
from datetime import datetime
import os
import json

# Загрузка конфигурации из файла
def load_config():
    """Загружает конфигурацию из JSON файла"""
    try:
        with open('config.json', 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print("Файл config.json не найден. Используются настройки по умолчанию.")
        return get_default_config()
    except json.JSONDecodeError as e:
        print(f"Ошибка чтения config.json: {e}. Используются настройки по умолчанию.")
        return get_default_config()

def get_default_config():
    """Возвращает конфигурацию по умолчанию"""
    return {
        "logging": {
            "enabled": True,
            "level": "INFO",
            "log_to_file": True,
            "log_to_console": True,
            "log_directory": "logs"
        },
        "api": {
            "key": "cdbda41e-e8e4-4e8d-9886-f12c075adc64",
            "timeout": 10,
            "sleep_between_requests": 0.5
        },
        "data_collection": {
            "page_size": 10,
            "max_pages": 5,
            "sleep_between_districts": 1
        }
    }

# Загружаем конфигурацию
CONFIG = load_config()


# Настройка логирования на основе конфига
def setup_logging():
    """Настройка системы логирования из конфигурационного файла"""

    if not CONFIG['logging']['enabled']:
        # Выключаем логирование
        logging.getLogger().setLevel(logging.CRITICAL)
        return

    # Создаем папку для логов если её нет
    log_dir = CONFIG['logging']['log_directory']
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # Преобразуем строковый уровень в числовой
    level_name = CONFIG['logging']['level']
    level = getattr(logging, level_name.upper(), logging.INFO)

    # Конфигурация логгера
    logger_config = {
        'version': 1,
        'disable_existing_loggers': False,
        'formatters': {
            'detailed': {
                'format': '%(asctime)s - %(levelname)s - [%(funcName)s] - %(message)s'
            },
            'simple': {
                'format': '%(levelname)s - %(message)s'
            }
        },
        'handlers': {
            'file': {
                'class': 'logging.FileHandler',
                'filename': f"{log_dir}/kindergartens_collection_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log",
                'formatter': 'detailed',
                'encoding': 'utf-8'
            },
            'console': {
                'class': 'logging.StreamHandler',
                'formatter': 'simple',
                'level': 'INFO'
            }
        },
        'root': {
            'level': level,
            'handlers': []
        }
    }

    # Добавляем обработчики в зависимости от настроек
    if CONFIG['logging']['log_to_file']:
        logger_config['root']['handlers'].append('file')
    if CONFIG['logging']['log_to_console']:
        logger_config['root']['handlers'].append('console')

    # Применяем конфигурацию
    logging.config.dictConfig(logger_config)

# Настраиваем логирование
setup_logging()

def get_kindergartens_in_district(district_id, district_name):
    """
    Функция для получения всех детских садов в указанном районе
    """
    logger = logging.getLogger(__name__)
    all_kindergartens = []
    page = 1
    page_size = CONFIG['data_collection']['page_size']
    max_pages = CONFIG['data_collection']['max_pages']

    logger.info(f"Начало сбора детских садов для района: {district_name} (ID: {district_id})")

    while page <= max_pages:
        params = {
            'key': CONFIG['api']['key'],
            'q': 'детский сад',
            'district_id': district_id,
            'type': 'branch',
            'page': page,
            'page_size': page_size,
            'fields': 'items.point,items.name,items.address_name,items.reviews'
        }

        url = "https://catalog.api.2gis.com/3.0/items"

        logger.debug(f"Запрос страницы {page} для района {district_name}")

        try:
            response = requests.get(url, params=params, timeout=CONFIG['api']['timeout'])
            logger.debug(f"Статус ответа: {response.status_code}")

            if response.status_code == 200:
                data = response.json()

                if 'result' in data and 'items' in data['result']:
                    kindergartens = data['result']['items']

                    if kindergartens:
                        logger.info(f"Загружено {len(kindergartens)} детских садов со страницы {page}")
                        for kindergarten in kindergartens:
                            kindergarten['district_name'] = district_name
                        all_kindergartens.extend(kindergartens)
                    else:
                        logger.debug(f"Страница {page} пуста")
                else:
                    logger.warning(f"На странице {page} нет данных в ожидаемом формате")

                page += 1

            else:
                logger.error(f"HTTP ошибка: {response.status_code} для страницы {page}")
                page += 1

        except requests.exceptions.Timeout:
            logger.error(f"Таймаут запроса на странице {page}")
            page += 1
        except Exception as e:
            logger.error(f"Ошибка на странице {page}: {e}")
            page += 1

        if page <= max_pages:
            time.sleep(CONFIG['api']['sleep_between_requests'])

    logger.info(f"Завершение сбора для района {district_name}. Найдено детских садов: {len(all_kindergartens)}")
    return all_kindergartens

def create_kindergartens_dataset(all_kindergartens):
    """
    Создает структурированный датасет из собранных данных о детских садах
    """
    logger = logging.getLogger(__name__)
    logger.info(f"Создание датасета из {len(all_kindergartens)} детских садов")

    data = []

    for kindergarten in all_kindergartens:
        reviews_data = kindergarten.get('reviews', {})
        point_data = kindergarten.get('point', {})

        row = {
            'district': kindergarten.get('district_name', ''),
            'type': 'детский сад',
            'name': kindergarten.get('name', ''),
            'address': kindergarten.get('address_name', ''),
            'latitude': point_data.get('lat', ''),
            'longitude': point_data.get('lon', ''),
            'rating': reviews_data.get('general_rating', 'нет рейтинга'),
            'reviews_count': reviews_data.get('general_review_count', 0),
            'ratings_count': reviews_data.get('general_review_count_with_stars', 0)
        }
        data.append(row)

    logger.info(f"Датасет детских садов создан. Количество строк: {len(data)}")
    return pd.DataFrame(data)

# Основной код
if __name__ == "__main__":
    logger = logging.getLogger(__name__)

    logger.info("=" * 60)
    logger.info("ЗАПУСК СБОРА ДАННЫХ ПО ДЕТСКИМ САДАМ МОСКВЫ")
    logger.info(f"Конфигурация логирования: уровень={CONFIG['logging']['level']}, файл={CONFIG['logging']['log_to_file']}, консоль={CONFIG['logging']['log_to_console']}")
    logger.info(f"Количество районов для обработки: {len(DISTRICT_IDS)}")
    logger.info("=" * 60)

    all_kindergartens_moscow = []
    start_time = time.time()

    # Обрабатываем каждый район
    for i, (district_name, district_id) in enumerate(DISTRICT_IDS.items(), 1):
        logger.info(f"[{i}/{len(DISTRICT_IDS)}] Обработка района: {district_name}")

        try:
            kindergartens_in_district = get_kindergartens_in_district(district_id, district_name)
            all_kindergartens_moscow.extend(kindergartens_in_district)
            logger.info(f"Район '{district_name}' завершен. Детских садов: {len(kindergartens_in_district)}")

        except Exception as e:
            logger.error(f"Ошибка в районе {district_name}: {e}")
            continue

        if i < len(DISTRICT_IDS):
            time.sleep(CONFIG['data_collection']['sleep_between_districts'])

    # Финализация
    end_time = time.time()
    execution_time = end_time - start_time

    logger.info("=" * 60)
    logger.info(f"СБОР ДАННЫХ ЗАВЕРШЕН")
    logger.info(f"Время выполнения: {execution_time:.2f} секунд")
    logger.info(f"Всего собрано детских садов: {len(all_kindergartens_moscow)}")
    logger.info("=" * 60)

    # Создаем датасет
    if all_kindergartens_moscow:
        df4 = create_kindergartens_dataset(all_kindergartens_moscow)
        filename = 'детские_сады_москвы_по_районам.csv'
        df4.to_csv(filename, index=False, encoding='utf-8')
        logger.info(f"Данные сохранены в {filename}")

        logger.info(f"СТАТИСТИКА: Всего детских садов: {len(df4)}")
        logger.info(f"СТАТИСТИКА: Охвачено районов: {df4['district'].nunique()}")

    else:
        logger.error("Не удалось собрать данные о детских садах")
        print("Не удалось собрать данные о детских садах")

In [ ]:
df4

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Академический,детский сад,Дошкольное отделение,"улица Ивана Бабушкина, 6",55.688590,37.564282,нет рейтинга,0,0
1,Академический,детский сад,Школа №1280,"улица Кржижановского, 30",55.675142,37.579230,5,2,4
2,Академический,детский сад,Детский сад,"улица Кедрова, 22 к1",55.680919,37.575902,нет рейтинга,0,1
3,Академический,детский сад,Детский сад,"улица Кржижановского, 28 к2",55.675908,37.578310,нет рейтинга,0,4
4,Академический,детский сад,Дошкольное отделение,"60-летия Октября проспект, 16 к5",55.690962,37.571719,3.6,3,7
...,...,...,...,...,...,...,...,...,...
3572,Ясенево,детский сад,"Ясенево, детский сад","улица Рокотова, 8 к3",55.611870,37.547997,нет рейтинга,0,3
3573,Ясенево,детский сад,"Дошкольное отделение, корпус №6","улица Айвазовского, 6 к3",55.616553,37.536502,нет рейтинга,0,1
3574,Ясенево,детский сад,"Дошкольное отделение, корпус №8","Новоясеневский проспект, 3 к2",55.608614,37.528703,нет рейтинга,0,1
3575,Ясенево,детский сад,Дошкольное отделение,"Голубинская улица, 7 к3",55.605360,37.513530,1,1,1


In [ ]:
len(df4['район'].value_counts())

132

Во всех районах Москвы есть детские сады.

#### Магазины (продукты, супермаркеты)

Берем новый ключ, чтобы не прервалось.

In [ ]:
API_KEY = "18ee47f5-2184-45df-ac89-779be9385d71"

In [ ]:
import requests
import time
import pandas as pd
import logging
import logging.config
from datetime import datetime
import os
import json

# Загрузка конфигурации из файла
def load_config():
    """Загружает конфигурацию из JSON файла"""
    try:
        with open('config.json', 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print("Файл config.json не найден. Используются настройки по умолчанию.")
        return get_default_config()
    except json.JSONDecodeError as e:
        print(f"Ошибка чтения config.json: {e}. Используются настройки по умолчанию.")
        return get_default_config()

def get_default_config():
    """Возвращает конфигурацию по умолчанию"""
    return {
        "logging": {
            "enabled": True,
            "level": "INFO",
            "log_to_file": True,
            "log_to_console": True,
            "log_directory": "logs"
        },
        "api": {
            "key": "18ee47f5-2184-45df-ac89-779be9385d71",
            "timeout": 10,
            "sleep_between_requests": 0.5
        },
        "data_collection": {
            "page_size": 10,
            "max_pages": 5,
            "sleep_between_districts": 1
        }
    }

# Загружаем конфигурацию
CONFIG = load_config()


# Настройка логирования на основе конфига
def setup_logging():
    """Настройка системы логирования из конфигурационного файла"""

    if not CONFIG['logging']['enabled']:
        # Выключаем логирование
        logging.getLogger().setLevel(logging.CRITICAL)
        return

    # Создаем папку для логов если её нет
    log_dir = CONFIG['logging']['log_directory']
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # Преобразуем строковый уровень в числовой
    level_name = CONFIG['logging']['level']
    level = getattr(logging, level_name.upper(), logging.INFO)

    # Конфигурация логгера
    logger_config = {
        'version': 1,
        'disable_existing_loggers': False,
        'formatters': {
            'detailed': {
                'format': '%(asctime)s - %(levelname)s - [%(funcName)s] - %(message)s'
            },
            'simple': {
                'format': '%(levelname)s - %(message)s'
            }
        },
        'handlers': {
            'file': {
                'class': 'logging.FileHandler',
                'filename': f"{log_dir}/supermarkets_collection_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log",
                'formatter': 'detailed',
                'encoding': 'utf-8'
            },
            'console': {
                'class': 'logging.StreamHandler',
                'formatter': 'simple',
                'level': 'INFO'
            }
        },
        'root': {
            'level': level,
            'handlers': []
        }
    }

    # Добавляем обработчики в зависимости от настроек
    if CONFIG['logging']['log_to_file']:
        logger_config['root']['handlers'].append('file')
    if CONFIG['logging']['log_to_console']:
        logger_config['root']['handlers'].append('console')

    # Применяем конфигурацию
    logging.config.dictConfig(logger_config)

# Настраиваем логирование
setup_logging()

def get_shops_in_district(district_id, district_name):
    """
    Функция для получения всех магазинов в указанном районе
    """
    logger = logging.getLogger(__name__)
    all_shops = []
    page = 1
    page_size = CONFIG['data_collection']['page_size']
    max_pages = CONFIG['data_collection']['max_pages']

    logger.info(f"Начало сбора магазинов для района: {district_name} (ID: {district_id})")

    while page <= max_pages:
        params = {
            'key': CONFIG['api']['key'],
            'q': 'магазин',
            'district_id': district_id,
            'type': 'branch',
            'page': page,
            'page_size': page_size,
            'fields': 'items.point,items.name,items.address_name,items.reviews'
        }

        url = "https://catalog.api.2gis.com/3.0/items"

        logger.debug(f"Запрос страницы {page} для района {district_name}")

        try:
            response = requests.get(url, params=params, timeout=CONFIG['api']['timeout'])
            logger.debug(f"Статус ответа: {response.status_code}")

            if response.status_code == 200:
                data = response.json()

                if 'result' in data and 'items' in data['result']:
                    shops = data['result']['items']

                    if shops:
                        logger.info(f"Загружено {len(shops)} магазинов со страницы {page}")
                        for shop in shops:
                            shop['district_name'] = district_name
                        all_shops.extend(shops)
                    else:
                        logger.debug(f"Страница {page} пуста")
                else:
                    logger.warning(f"На странице {page} нет данных в ожидаемом формате")

                page += 1

            else:
                logger.error(f"HTTP ошибка: {response.status_code} для страницы {page}")
                page += 1

        except requests.exceptions.Timeout:
            logger.error(f"Таймаут запроса на странице {page}")
            page += 1
        except Exception as e:
            logger.error(f"Ошибка на странице {page}: {e}")
            page += 1

        if page <= max_pages:
            time.sleep(CONFIG['api']['sleep_between_requests'])

    logger.info(f"Завершение сбора для района {district_name}. Найдено магазинов: {len(all_shops)}")
    return all_shops

def create_shops_dataset(all_shops):
    """
    Создает структурированный датасет из собранных данных о магазинах
    """
    logger = logging.getLogger(__name__)
    logger.info(f"Создание датасета из {len(all_shops)} магазинов")

    data = []

    for shop in all_shops:
        reviews_data = shop.get('reviews', {})
        point_data = shop.get('point', {})

        row = {
            'district': shop.get('district_name', ''),
            'type': 'магазин',
            'name': shop.get('name', ''),
            'address': shop.get('address_name', ''),
            'latitude': point_data.get('lat', ''),
            'longitude': point_data.get('lon', ''),
            'rating': reviews_data.get('general_rating', 'нет рейтинга'),
            'reviews_count': reviews_data.get('general_review_count', 0),
            'ratings_count': reviews_data.get('general_review_count_with_stars', 0)
        }
        data.append(row)

    logger.info(f"Датасет магазинов создан. Количество строк: {len(data)}")
    return pd.DataFrame(data)

# Основной код
if __name__ == "__main__":
    logger = logging.getLogger(__name__)

    logger.info("=" * 60)
    logger.info("ЗАПУСК СБОРА ДАННЫХ ПО МАГАЗИНАМ МОСКВЫ")
    logger.info(f"Конфигурация логирования: уровень={CONFIG['logging']['level']}, файл={CONFIG['logging']['log_to_file']}, консоль={CONFIG['logging']['log_to_console']}")
    logger.info(f"Количество районов для обработки: {len(DISTRICT_IDS)}")
    logger.info("=" * 60)

    all_shops_moscow = []
    start_time = time.time()

    # Обрабатываем каждый район
    for i, (district_name, district_id) in enumerate(DISTRICT_IDS.items(), 1):
        logger.info(f"[{i}/{len(DISTRICT_IDS)}] Обработка района: {district_name}")

        try:
            shops_in_district = get_shops_in_district(district_id, district_name)
            all_shops_moscow.extend(shops_in_district)
            logger.info(f"Район '{district_name}' завершен. Магазинов: {len(shops_in_district)}")

        except Exception as e:
            logger.error(f"Ошибка в районе {district_name}: {e}")
            continue

        if i < len(DISTRICT_IDS):
            time.sleep(CONFIG['data_collection']['sleep_between_districts'])

    # Финализация
    end_time = time.time()
    execution_time = end_time - start_time

    logger.info("=" * 60)
    logger.info(f"СБОР ДАННЫХ ЗАВЕРШЕН")
    logger.info(f"Время выполнения: {execution_time:.2f} секунд")
    logger.info(f"Всего собрано магазинов: {len(all_shops_moscow)}")
    logger.info("=" * 60)

    # Создаем датасет
    if all_shops_moscow:
        df5 = create_shops_dataset(all_shops_moscow)
        filename = 'магазины_москвы_по_районам.csv'
        df5.to_csv(filename, index=False, encoding='utf-8')
        logger.info(f"Данные сохранены в {filename}")

        logger.info(f"СТАТИСТИКА: Всего магазинов: {len(df5)}")
        logger.info(f"СТАТИСТИКА: Охвачено районов: {df5['district'].nunique()}")

    else:
        logger.error("Не удалось собрать данные о магазинах")
        print("Не удалось собрать данные о магазинах")

In [ ]:
df5

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Академический,магазин,"Перекрёсток, супермаркет","улица Кедрова, 7а",55.685454,37.568587,4.7,54,142
1,Академический,магазин,"Перекрёсток, супермаркет","Новочерёмушкинская улица, 26/1",55.680029,37.575315,4.1,24,46
2,Академический,магазин,"Пятёрочка, супермаркет","улица Дмитрия Ульянова, 6",55.690423,37.563613,4.6,89,381
3,Академический,магазин,"Супер Лента, супермаркет","улица Винокурова, 4",55.689294,37.582606,4.6,59,266
4,Академический,магазин,"Пятёрочка, супермаркет","улица Гримау, 10а ст1",55.690010,37.578720,4.7,53,204
...,...,...,...,...,...,...,...,...,...
6447,Ясенево,магазин,"Авокадо, магазин продуктов","улица Айвазовского, 2",55.619308,37.541331,нет рейтинга,0,1
6448,Ясенево,магазин,"ВкусВилл, супермаркет","Тарусская улица, 14 к1",55.605209,37.530891,3.7,3,4
6449,Ясенево,магазин,"Mister Dom, магазин","Новоясеневский проспект, 1",55.619185,37.509856,нет рейтинга,0,0
6450,Ясенево,магазин,"Пятёрочка, супермаркет","проезд Одоевского, 11 к7",55.598310,37.512693,4.8,28,123


In [ ]:
len(df5['район'].value_counts())

132

Во всех районах Москвы есть магазины (продуктовые).

#### Кафе

Снова берем новый ключ.

In [ ]:
API_KEY = "50e21a4a-6241-4f9a-aff6-0b7a942be2b7"

In [ ]:
import requests
import time
import pandas as pd
import logging
import logging.config
from datetime import datetime
import os
import json

# Загрузка конфигурации из файла
def load_config():
    """Загружает конфигурацию из JSON файла"""
    try:
        with open('config.json', 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print("Файл config.json не найден. Используются настройки по умолчанию.")
        return get_default_config()
    except json.JSONDecodeError as e:
        print(f"Ошибка чтения config.json: {e}. Используются настройки по умолчанию.")
        return get_default_config()

def get_default_config():
    """Возвращает конфигурацию по умолчанию"""
    return {
        "logging": {
            "enabled": True,
            "level": "INFO",
            "log_to_file": True,
            "log_to_console": True,
            "log_directory": "logs"
        },
        "api": {
            "key": "50e21a4a-6241-4f9a-aff6-0b7a942be2b7",
            "timeout": 10,
            "sleep_between_requests": 0.5
        },
        "data_collection": {
            "page_size": 10,
            "max_pages": 5,
            "sleep_between_districts": 1
        }
    }

# Загружаем конфигурацию
CONFIG = load_config()


# Настройка логирования на основе конфига
def setup_logging():
    """Настройка системы логирования из конфигурационного файла"""

    if not CONFIG['logging']['enabled']:
        # Выключаем логирование
        logging.getLogger().setLevel(logging.CRITICAL)
        return

    # Создаем папку для логов если её нет
    log_dir = CONFIG['logging']['log_directory']
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # Преобразуем строковый уровень в числовой
    level_name = CONFIG['logging']['level']
    level = getattr(logging, level_name.upper(), logging.INFO)

    # Конфигурация логгера
    logger_config = {
        'version': 1,
        'disable_existing_loggers': False,
        'formatters': {
            'detailed': {
                'format': '%(asctime)s - %(levelname)s - [%(funcName)s] - %(message)s'
            },
            'simple': {
                'format': '%(levelname)s - %(message)s'
            }
        },
        'handlers': {
            'file': {
                'class': 'logging.FileHandler',
                'filename': f"{log_dir}/cafes_collection_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log",
                'formatter': 'detailed',
                'encoding': 'utf-8'
            },
            'console': {
                'class': 'logging.StreamHandler',
                'formatter': 'simple',
                'level': 'INFO'
            }
        },
        'root': {
            'level': level,
            'handlers': []
        }
    }

    # Добавляем обработчики в зависимости от настроек
    if CONFIG['logging']['log_to_file']:
        logger_config['root']['handlers'].append('file')
    if CONFIG['logging']['log_to_console']:
        logger_config['root']['handlers'].append('console')

    # Применяем конфигурацию
    logging.config.dictConfig(logger_config)

# Настраиваем логирование
setup_logging()

def get_cafes_in_district(district_id, district_name):
    """
    Функция для получения всех кафе в указанном районе
    """
    logger = logging.getLogger(__name__)
    all_cafes = []
    page = 1
    page_size = CONFIG['data_collection']['page_size']
    max_pages = CONFIG['data_collection']['max_pages']

    logger.info(f"Начало сбора кафе для района: {district_name} (ID: {district_id})")

    while page <= max_pages:
        params = {
            'key': CONFIG['api']['key'],
            'q': 'кафе',
            'district_id': district_id,
            'type': 'branch',
            'page': page,
            'page_size': page_size,
            'fields': 'items.point,items.name,items.address_name,items.reviews'
        }

        url = "https://catalog.api.2gis.com/3.0/items"

        logger.debug(f"Запрос страницы {page} для района {district_name}")

        try:
            response = requests.get(url, params=params, timeout=CONFIG['api']['timeout'])
            logger.debug(f"Статус ответа: {response.status_code}")

            if response.status_code == 200:
                data = response.json()

                if 'result' in data and 'items' in data['result']:
                    cafes = data['result']['items']

                    if cafes:
                        logger.info(f"Загружено {len(cafes)} кафе со страницы {page}")
                        for cafe in cafes:
                            cafe['district_name'] = district_name
                        all_cafes.extend(cafes)
                    else:
                        logger.debug(f"Страница {page} пуста")
                else:
                    logger.warning(f"На странице {page} нет данных в ожидаемом формате")

                page += 1

            else:
                logger.error(f"HTTP ошибка: {response.status_code} для страницы {page}")
                page += 1

        except requests.exceptions.Timeout:
            logger.error(f"Таймаут запроса на странице {page}")
            page += 1
        except Exception as e:
            logger.error(f"Ошибка на странице {page}: {e}")
            page += 1

        if page <= max_pages:
            time.sleep(CONFIG['api']['sleep_between_requests'])

    logger.info(f"Завершение сбора для района {district_name}. Найдено кафе: {len(all_cafes)}")
    return all_cafes

def create_cafes_dataset(all_cafes):
    """
    Создает структурированный датасет из собранных данных о кафе
    """
    logger = logging.getLogger(__name__)
    logger.info(f"Создание датасета из {len(all_cafes)} кафе")

    data = []

    for cafe in all_cafes:
        reviews_data = cafe.get('reviews', {})
        point_data = cafe.get('point', {})

        row = {
            'district': cafe.get('district_name', ''),
            'type': 'кафе',
            'name': cafe.get('name', ''),
            'address': cafe.get('address_name', ''),
            'latitude': point_data.get('lat', ''),
            'longitude': point_data.get('lon', ''),
            'rating': reviews_data.get('general_rating', 'нет рейтинга'),
            'reviews_count': reviews_data.get('general_review_count', 0),
            'ratings_count': reviews_data.get('general_review_count_with_stars', 0)
        }
        data.append(row)

    logger.info(f"Датасет кафе создан. Количество строк: {len(data)}")
    return pd.DataFrame(data)

# Основной код
if __name__ == "__main__":
    logger = logging.getLogger(__name__)

    logger.info("=" * 60)
    logger.info("ЗАПУСК СБОРА ДАННЫХ ПО КАФЕ МОСКВЫ")
    logger.info(f"Конфигурация логирования: уровень={CONFIG['logging']['level']}, файл={CONFIG['logging']['log_to_file']}, консоль={CONFIG['logging']['log_to_console']}")
    logger.info(f"Количество районов для обработки: {len(DISTRICT_IDS)}")
    logger.info("=" * 60)

    all_cafes_moscow = []
    start_time = time.time()

    # Обрабатываем каждый район
    for i, (district_name, district_id) in enumerate(DISTRICT_IDS.items(), 1):
        logger.info(f"[{i}/{len(DISTRICT_IDS)}] Обработка района: {district_name}")

        try:
            cafes_in_district = get_cafes_in_district(district_id, district_name)
            all_cafes_moscow.extend(cafes_in_district)
            logger.info(f"Район '{district_name}' завершен. Кафе: {len(cafes_in_district)}")

        except Exception as e:
            logger.error(f"Ошибка в районе {district_name}: {e}")
            continue

        if i < len(DISTRICT_IDS):
            time.sleep(CONFIG['data_collection']['sleep_between_districts'])

    # Финализация
    end_time = time.time()
    execution_time = end_time - start_time

    logger.info("=" * 60)
    logger.info(f"СБОР ДАННЫХ ЗАВЕРШЕН")
    logger.info(f"Время выполнения: {execution_time:.2f} секунд")
    logger.info(f"Всего собрано кафе: {len(all_cafes_moscow)}")
    logger.info("=" * 60)

    # Создаем датасет
    if all_cafes_moscow:
        df6 = create_cafes_dataset(all_cafes_moscow)
        filename = 'кафе_москвы_по_районам.csv'
        df6.to_csv(filename, index=False, encoding='utf-8')
        logger.info(f"Данные сохранены в {filename}")

        logger.info(f"СТАТИСТИКА: Всего кафе: {len(df6)}")
        logger.info(f"СТАТИСТИКА: Охвачено районов: {df6['district'].nunique()}")

    else:
        logger.error("Не удалось собрать данные о кафе")
        print("Не удалось собрать данные о кафе")

In [ ]:
df6

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Академический,кафе,"Jolly leprechaun, паб","60-летия Октября проспект, 20",55.688456,37.573570,4.7,201,343
1,Академический,кафе,Чисто Ростовский Бар,"улица Кржижановского, 20/30 к1",55.679105,37.572008,5,9,19
2,Академический,кафе,"Шоколадница, кофейня","Профсоюзная улица, 19",55.678722,37.564499,3.7,115,153
3,Академический,кафе,"Бухара, кафе","60-летия Октября проспект, 23",55.691761,37.576367,4,27,46
4,Академический,кафе,"Здрасте, кофейня","Новочерёмушкинская улица, 17",55.682888,37.583401,4.8,26,28
...,...,...,...,...,...,...,...,...,...
3969,Ясенево,кафе,"Istanbul shef doner, кафе","Профсоюзная улица, 129а",55.618757,37.506383,1.9,7,9
3970,Ясенево,кафе,"Крошка картошка, кафе","Новоясеневский проспект, 11",55.606282,37.536142,4.1,12,48
3971,Ясенево,кафе,"Дагестанская кухня, кафе","Профсоюзная улица, 129а",55.617892,37.508103,5,1,1
3972,Ясенево,кафе,"ВкусВилл, кафе","Новоясеневский проспект, 1 к2",55.619800,37.507233,нет рейтинга,0,3


In [ ]:
len(df6['район'].value_counts())

131

Не в каждом районе Москвы есть кафе, а именно, кафе нет только в одном районе.

#### МУЗЕИ

Берем новый ключ.

In [ ]:
API_KEY = "c446e160-f8e9-420d-abcd-390476aa0e9f"

In [ ]:
import requests
import time
import pandas as pd
import logging
import logging.config
from datetime import datetime
import os
import json

# Загрузка конфигурации из файла
def load_config():
    """Загружает конфигурацию из JSON файла"""
    try:
        with open('config.json', 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print("Файл config.json не найден. Используются настройки по умолчанию.")
        return get_default_config()
    except json.JSONDecodeError as e:
        print(f"Ошибка чтения config.json: {e}. Используются настройки по умолчанию.")
        return get_default_config()

def get_default_config():
    """Возвращает конфигурацию по умолчанию"""
    return {
        "logging": {
            "enabled": True,
            "level": "INFO",
            "log_to_file": True,
            "log_to_console": True,
            "log_directory": "logs"
        },
        "api": {
            "key": "c446e160-f8e9-420d-abcd-390476aa0e9f",
            "timeout": 10,
            "sleep_between_requests": 0.5
        },
        "data_collection": {
            "page_size": 10,
            "max_pages": 5,
            "sleep_between_districts": 1
        }
    }

# Загружаем конфигурацию
CONFIG = load_config()


# Настройка логирования на основе конфига
def setup_logging():
    """Настройка системы логирования из конфигурационного файла"""

    if not CONFIG['logging']['enabled']:
        # Выключаем логирование
        logging.getLogger().setLevel(logging.CRITICAL)
        return

    # Создаем папку для логов если её нет
    log_dir = CONFIG['logging']['log_directory']
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # Преобразуем строковый уровень в числовой
    level_name = CONFIG['logging']['level']
    level = getattr(logging, level_name.upper(), logging.INFO)

    # Конфигурация логгера
    logger_config = {
        'version': 1,
        'disable_existing_loggers': False,
        'formatters': {
            'detailed': {
                'format': '%(asctime)s - %(levelname)s - [%(funcName)s] - %(message)s'
            },
            'simple': {
                'format': '%(levelname)s - %(message)s'
            }
        },
        'handlers': {
            'file': {
                'class': 'logging.FileHandler',
                'filename': f"{log_dir}/museums_collection_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log",
                'formatter': 'detailed',
                'encoding': 'utf-8'
            },
            'console': {
                'class': 'logging.StreamHandler',
                'formatter': 'simple',
                'level': 'INFO'
            }
        },
        'root': {
            'level': level,
            'handlers': []
        }
    }

    # Добавляем обработчики в зависимости от настроек
    if CONFIG['logging']['log_to_file']:
        logger_config['root']['handlers'].append('file')
    if CONFIG['logging']['log_to_console']:
        logger_config['root']['handlers'].append('console')

    # Применяем конфигурацию
    logging.config.dictConfig(logger_config)

# Настраиваем логирование
setup_logging()

def get_museums_in_district(district_id, district_name):
    """
    Функция для получения всех музеев в указанном районе
    """
    logger = logging.getLogger(__name__)
    all_museums = []
    page = 1
    page_size = CONFIG['data_collection']['page_size']
    max_pages = CONFIG['data_collection']['max_pages']

    logger.info(f"Начало сбора музеев для района: {district_name} (ID: {district_id})")

    while page <= max_pages:
        params = {
            'key': CONFIG['api']['key'],
            'q': 'музей',
            'district_id': district_id,
            'type': 'branch',
            'page': page,
            'page_size': page_size,
            'fields': 'items.point,items.name,items.address_name,items.reviews'
        }

        url = "https://catalog.api.2gis.com/3.0/items"

        logger.debug(f"Запрос страницы {page} для района {district_name}")

        try:
            response = requests.get(url, params=params, timeout=CONFIG['api']['timeout'])
            logger.debug(f"Статус ответа: {response.status_code}")

            if response.status_code == 200:
                data = response.json()

                if 'result' in data and 'items' in data['result']:
                    museums = data['result']['items']

                    if museums:
                        logger.info(f"Загружено {len(museums)} музеев со страницы {page}")
                        for museum in museums:
                            museum['district_name'] = district_name
                        all_museums.extend(museums)
                    else:
                        logger.debug(f"Страница {page} пуста")
                else:
                    logger.warning(f"На странице {page} нет данных в ожидаемом формате")

                page += 1

            else:
                logger.error(f"HTTP ошибка: {response.status_code} для страницы {page}")
                page += 1

        except requests.exceptions.Timeout:
            logger.error(f"Таймаут запроса на странице {page}")
            page += 1
        except Exception as e:
            logger.error(f"Ошибка на странице {page}: {e}")
            page += 1

        if page <= max_pages:
            time.sleep(CONFIG['api']['sleep_between_requests'])

    logger.info(f"Завершение сбора для района {district_name}. Найдено музеев: {len(all_museums)}")
    return all_museums

def create_museums_dataset(all_museums):
    """
    Создает структурированный датасет из собранных данных о музеях
    """
    logger = logging.getLogger(__name__)
    logger.info(f"Создание датасета из {len(all_museums)} музеев")

    data = []

    for museum in all_museums:
        reviews_data = museum.get('reviews', {})
        point_data = museum.get('point', {})

        row = {
            'district': museum.get('district_name', ''),
            'type': 'музей',
            'name': museum.get('name', ''),
            'address': museum.get('address_name', ''),
            'latitude': point_data.get('lat', ''),
            'longitude': point_data.get('lon', ''),
            'rating': reviews_data.get('general_rating', 'нет рейтинга'),
            'reviews_count': reviews_data.get('general_review_count', 0),
            'ratings_count': reviews_data.get('general_review_count_with_stars', 0)
        }
        data.append(row)

    logger.info(f"Датасет музеев создан. Количество строк: {len(data)}")
    return pd.DataFrame(data)

# Основной код
if __name__ == "__main__":
    logger = logging.getLogger(__name__)

    logger.info("=" * 60)
    logger.info("ЗАПУСК СБОРА ДАННЫХ ПО МУЗЕЯМ МОСКВЫ")
    logger.info(f"Конфигурация логирования: уровень={CONFIG['logging']['level']}, файл={CONFIG['logging']['log_to_file']}, консоль={CONFIG['logging']['log_to_console']}")
    logger.info(f"Количество районов для обработки: {len(DISTRICT_IDS)}")
    logger.info("=" * 60)

    all_museums_moscow = []
    start_time = time.time()

    # Обрабатываем каждый район
    for i, (district_name, district_id) in enumerate(DISTRICT_IDS.items(), 1):
        logger.info(f"[{i}/{len(DISTRICT_IDS)}] Обработка района: {district_name}")

        try:
            museums_in_district = get_museums_in_district(district_id, district_name)
            all_museums_moscow.extend(museums_in_district)
            logger.info(f"Район '{district_name}' завершен. Музеев: {len(museums_in_district)}")

        except Exception as e:
            logger.error(f"Ошибка в районе {district_name}: {e}")
            continue

        if i < len(DISTRICT_IDS):
            time.sleep(CONFIG['data_collection']['sleep_between_districts'])

    # Финализация
    end_time = time.time()
    execution_time = end_time - start_time

    logger.info("=" * 60)
    logger.info(f"СБОР ДАННЫХ ЗАВЕРШЕН")
    logger.info(f"Время выполнения: {execution_time:.2f} секунд")
    logger.info(f"Всего собрано музеев: {len(all_museums_moscow)}")
    logger.info("=" * 60)

    # Создаем датасет
    if all_museums_moscow:
        df7 = create_museums_dataset(all_museums_moscow)
        filename = 'музеи_москвы_по_районам.csv'
        df7.to_csv(filename, index=False, encoding='utf-8')
        logger.info(f"Данные сохранены в {filename}")

        logger.info(f"СТАТИСТИКА: Всего музеев: {len(df7)}")
        logger.info(f"СТАТИСТИКА: Охвачено районов: {df7['district'].nunique()}")

    else:
        logger.error("Не удалось собрать данные о музеях")
        print("Не удалось собрать данные о музеях")

In [ ]:
df7

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Академический,музей,Государственный Дарвиновский музей,"улица Вавилова, 57",55.690628,37.561520,4.8,571,1808
1,Академический,музей,Государственный Дарвиновский музей,"улица Вавилова, 57 ст1",55.690843,37.562429,4.7,142,348
2,Академический,музей,"Чайникландия, музей удивительных чайников мира","улица Винокурова, 7/5 к3",55.688856,37.587414,4.9,9,12
3,Академический,музей,Музей истории Мосэнерго,"улица Вавилова, 13 ст5",55.702586,37.582913,4.7,4,9
4,Академический,музей,Музей Российского Красного Креста,"Черёмушкинский проезд, 5",55.691891,37.572537,4,1,1
...,...,...,...,...,...,...,...,...,...
597,Якиманка,музей,"МИСиС, музей","Ленинский проспект, 6 ст1",55.727098,37.607079,нет рейтинга,0,0
598,Якиманка,музей,"Кадашевская слобода, канцелярия","Кадашёвский тупик, 2 ст6",55.743082,37.623145,1.9,1,5
599,Якиманка,музей,Храм иконы Божией Матери всех скорбящих радост...,"3-й Кадашёвский переулок, 3 ст6",55.741702,37.624128,5,2,3
600,Якиманка,музей,"Городская усадьба Масленниковых-Петуховых, г. ...","Щетининский переулок, 10 ст1",55.733767,37.622883,нет рейтинга,0,1


In [ ]:
len(df7['район'].value_counts())

95

Далеко не во всех районах Москвы есть музеи.

#### Торговые центры

In [ ]:
API_KEY = "5bac6b44-d0d1-4d25-bd9a-1a7b91697840"

In [ ]:
import requests
import time
import pandas as pd
import logging
import logging.config
from datetime import datetime
import os
import json

# Загрузка конфигурации из файла
def load_config():
    """Загружает конфигурацию из JSON файла"""
    try:
        with open('config.json', 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print("Файл config.json не найден. Используются настройки по умолчанию.")
        return get_default_config()
    except json.JSONDecodeError as e:
        print(f"Ошибка чтения config.json: {e}. Используются настройки по умолчанию.")
        return get_default_config()

def get_default_config():
    """Возвращает конфигурацию по умолчанию"""
    return {
        "logging": {
            "enabled": True,
            "level": "INFO",
            "log_to_file": True,
            "log_to_console": True,
            "log_directory": "logs"
        },
        "api": {
            "key": "5bac6b44-d0d1-4d25-bd9a-1a7b91697840",
            "timeout": 10,
            "sleep_between_requests": 0.5
        },
        "data_collection": {
            "page_size": 10,
            "max_pages": 5,
            "sleep_between_districts": 1
        }
    }

# Загружаем конфигурацию
CONFIG = load_config()


# Настройка логирования на основе конфига
def setup_logging():
    """Настройка системы логирования из конфигурационного файла"""

    if not CONFIG['logging']['enabled']:
        # Выключаем логирование
        logging.getLogger().setLevel(logging.CRITICAL)
        return

    # Создаем папку для логов если её нет
    log_dir = CONFIG['logging']['log_directory']
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # Преобразуем строковый уровень в числовой
    level_name = CONFIG['logging']['level']
    level = getattr(logging, level_name.upper(), logging.INFO)

    # Конфигурация логгера
    logger_config = {
        'version': 1,
        'disable_existing_loggers': False,
        'formatters': {
            'detailed': {
                'format': '%(asctime)s - %(levelname)s - [%(funcName)s] - %(message)s'
            },
            'simple': {
                'format': '%(levelname)s - %(message)s'
            }
        },
        'handlers': {
            'file': {
                'class': 'logging.FileHandler',
                'filename': f"{log_dir}/torgcentre_collection_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log",
                'formatter': 'detailed',
                'encoding': 'utf-8'
            },
            'console': {
                'class': 'logging.StreamHandler',
                'formatter': 'simple',
                'level': 'INFO'
            }
        },
        'root': {
            'level': level,
            'handlers': []
        }
    }

    # Добавляем обработчики в зависимости от настроек
    if CONFIG['logging']['log_to_file']:
        logger_config['root']['handlers'].append('file')
    if CONFIG['logging']['log_to_console']:
        logger_config['root']['handlers'].append('console')

    # Применяем конфигурацию
    logging.config.dictConfig(logger_config)

# Настраиваем логирование
setup_logging()

def get_shopping_centers_in_district(district_id, district_name):
    """
    Функция для получения всех торговых центров в указанном районе
    """
    logger = logging.getLogger(__name__)
    all_shopping_centers = []
    page = 1
    page_size = CONFIG['data_collection']['page_size']
    max_pages = CONFIG['data_collection']['max_pages']

    logger.info(f"Начало сбора торговых центров для района: {district_name} (ID: {district_id})")

    while page <= max_pages:
        params = {
            'key': CONFIG['api']['key'],
            'q': 'торговый центр',
            'district_id': district_id,
            'type': 'branch',
            'page': page,
            'page_size': page_size,
            'fields': 'items.point,items.name,items.address_name,items.reviews'
        }

        url = "https://catalog.api.2gis.com/3.0/items"

        logger.debug(f"Запрос страницы {page} для района {district_name}")

        try:
            response = requests.get(url, params=params, timeout=CONFIG['api']['timeout'])
            logger.debug(f"Статус ответа: {response.status_code}")

            if response.status_code == 200:
                data = response.json()

                if 'result' in data and 'items' in data['result']:
                    shopping_centers = data['result']['items']

                    if shopping_centers:
                        logger.info(f"Загружено {len(shopping_centers)} торговых центров со страницы {page}")
                        for sc in shopping_centers:
                            sc['district_name'] = district_name
                        all_shopping_centers.extend(shopping_centers)
                    else:
                        logger.debug(f"Страница {page} пуста")
                else:
                    logger.warning(f"На странице {page} нет данных в ожидаемом формате")

                page += 1

            else:
                logger.error(f"HTTP ошибка: {response.status_code} для страницы {page}")
                page += 1

        except requests.exceptions.Timeout:
            logger.error(f"Таймаут запроса на странице {page}")
            page += 1
        except Exception as e:
            logger.error(f"Ошибка на странице {page}: {e}")
            page += 1

        if page <= max_pages:
            time.sleep(CONFIG['api']['sleep_between_requests'])

    logger.info(f"Завершение сбора для района {district_name}. Найдено торговых центров: {len(all_shopping_centers)}")
    return all_shopping_centers

def create_shopping_centers_dataset(all_shopping_centers):
    """
    Создает структурированный датасет из собранных данных о торговых центрах
    """
    logger = logging.getLogger(__name__)
    logger.info(f"Создание датасета из {len(all_shopping_centers)} торговых центров")

    data = []

    for sc in all_shopping_centers:
        reviews_data = sc.get('reviews', {})
        point_data = sc.get('point', {})

        row = {
            'district': sc.get('district_name', ''),
            'type': 'торговый центр',
            'name': sc.get('name', ''),
            'address': sc.get('address_name', ''),
            'latitude': point_data.get('lat', ''),
            'longitude': point_data.get('lon', ''),
            'rating': reviews_data.get('general_rating', 'нет рейтинга'),
            'reviews_count': reviews_data.get('general_review_count', 0),
            'ratings_count': reviews_data.get('general_review_count_with_stars', 0)
        }
        data.append(row)

    logger.info(f"Датасет торговых центров создан. Количество строк: {len(data)}")
    return pd.DataFrame(data)

# Основной код
if __name__ == "__main__":
    logger = logging.getLogger(__name__)

    logger.info("=" * 60)
    logger.info("ЗАПУСК СБОРА ДАННЫХ ПО ТОРГОВЫМ ЦЕНТРАМ МОСКВЫ")
    logger.info(f"Конфигурация логирования: уровень={CONFIG['logging']['level']}, файл={CONFIG['logging']['log_to_file']}, консоль={CONFIG['logging']['log_to_console']}")
    logger.info(f"Количество районов для обработки: {len(DISTRICT_IDS)}")
    logger.info("=" * 60)

    all_shopping_centers_moscow = []
    start_time = time.time()

    # Обрабатываем каждый район
    for i, (district_name, district_id) in enumerate(DISTRICT_IDS.items(), 1):
        logger.info(f"[{i}/{len(DISTRICT_IDS)}] Обработка района: {district_name}")

        try:
            shopping_centers_in_district = get_shopping_centers_in_district(district_id, district_name)
            all_shopping_centers_moscow.extend(shopping_centers_in_district)
            logger.info(f"Район '{district_name}' завершен. Торговых центров: {len(shopping_centers_in_district)}")

        except Exception as e:
            logger.error(f"Ошибка в районе {district_name}: {e}")
            continue

        if i < len(DISTRICT_IDS):
            time.sleep(CONFIG['data_collection']['sleep_between_districts'])

    # Финализация
    end_time = time.time()
    execution_time = end_time - start_time

    logger.info("=" * 60)
    logger.info(f"СБОР ДАННЫХ ЗАВЕРШЕН")
    logger.info(f"Время выполнения: {execution_time:.2f} секунд")
    logger.info(f"Всего собрано торговых центров: {len(all_shopping_centers_moscow)}")
    logger.info("=" * 60)

    # Создаем датасет
    if all_shopping_centers_moscow:
        df8 = create_shopping_centers_dataset(all_shopping_centers_moscow)
        filename = 'торговые_центры_москвы_по_районам.csv'
        df8.to_csv(filename, index=False, encoding='utf-8')
        logger.info(f"Данные сохранены в {filename}")

        logger.info(f"СТАТИСТИКА: Всего торговых центров: {len(df8)}")
        logger.info(f"СТАТИСТИКА: Охвачено районов: {df8['district'].nunique()}")

    else:
        logger.error("Не удалось собрать данные о торговых центрах")
        print("Не удалось собрать данные о торговых центрах")

In [ ]:
df8

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Академический,торговый центр,"Черёмушки, торговый центр","улица Шверника, 13 к1",55.691660,37.589307,4.2,8,59
1,Академический,торговый центр,"Олиан, торговый центр","Профсоюзная улица, 12",55.685445,37.570214,4.5,8,24
2,Алексеевский,торговый центр,"Алексеевский, торговый центр","проспект Мира, 114Б ст2",55.808243,37.637985,4.6,25,165
3,Алексеевский,торговый центр,"Колорит, торговый центр","Новоалексеевская улица, 12 ст1",55.808408,37.644744,5,4,22
4,Алексеевский,торговый центр,"Солярис, торговый дом","проспект Мира, 104 ст2",55.802223,37.637665,нет рейтинга,0,0
...,...,...,...,...,...,...,...,...,...
924,Ясенево,торговый центр,"Этажи, торговый центр","Ясногорская улица, 7а",55.608596,37.537380,4.5,5,40
925,Ясенево,торговый центр,"Кэмп, торговый центр","Вильнюсская улица, 1",55.606883,37.524839,3.4,7,26
926,Ясенево,торговый центр,"Спектр, торговый центр","Новоясеневский проспект, 1 к1",55.619452,37.506828,4.5,8,52
927,Ясенево,торговый центр,"Торгсин-маркет, торговый дом","улица Паустовского, 1",55.602706,37.542374,3.5,1,5


In [ ]:
len(df8['район'].value_counts())

127

Не во всех районах Москвы есть торговые центры.

#### Бизнес-центры

In [ ]:
API_KEY = "a3598e20-28b7-488f-b95b-d785f017ea56"

In [ ]:
import requests
import time
import pandas as pd
import logging
import logging.config
from datetime import datetime
import os
import json

# Загрузка конфигурации из файла
def load_config():
    """Загружает конфигурацию из JSON файла"""
    try:
        with open('config.json', 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print("Файл config.json не найден. Используются настройки по умолчанию.")
        return get_default_config()
    except json.JSONDecodeError as e:
        print(f"Ошибка чтения config.json: {e}. Используются настройки по умолчанию.")
        return get_default_config()

def get_default_config():
    """Возвращает конфигурацию по умолчанию"""
    return {
        "logging": {
            "enabled": True,
            "level": "INFO",
            "log_to_file": True,
            "log_to_console": True,
            "log_directory": "logs"
        },
        "api": {
            "key": "a3598e20-28b7-488f-b95b-d785f017ea56",
            "timeout": 10,
            "sleep_between_requests": 0.5
        },
        "data_collection": {
            "page_size": 10,
            "max_pages": 5,
            "sleep_between_districts": 1
        }
    }

# Загружаем конфигурацию
CONFIG = load_config()


# Настройка логирования на основе конфига
def setup_logging():
    """Настройка системы логирования из конфигурационного файла"""

    if not CONFIG['logging']['enabled']:
        # Выключаем логирование
        logging.getLogger().setLevel(logging.CRITICAL)
        return

    # Создаем папку для логов если её нет
    log_dir = CONFIG['logging']['log_directory']
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    # Преобразуем строковый уровень в числовой
    level_name = CONFIG['logging']['level']
    level = getattr(logging, level_name.upper(), logging.INFO)

    # Конфигурация логгера
    logger_config = {
        'version': 1,
        'disable_existing_loggers': False,
        'formatters': {
            'detailed': {
                'format': '%(asctime)s - %(levelname)s - [%(funcName)s] - %(message)s'
            },
            'simple': {
                'format': '%(levelname)s - %(message)s'
            }
        },
        'handlers': {
            'file': {
                'class': 'logging.FileHandler',
                'filename': f"{log_dir}/business_collection_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log",
                'formatter': 'detailed',
                'encoding': 'utf-8'
            },
            'console': {
                'class': 'logging.StreamHandler',
                'formatter': 'simple',
                'level': 'INFO'
            }
        },
        'root': {
            'level': level,
            'handlers': []
        }
    }

    # Добавляем обработчики в зависимости от настроек
    if CONFIG['logging']['log_to_file']:
        logger_config['root']['handlers'].append('file')
    if CONFIG['logging']['log_to_console']:
        logger_config['root']['handlers'].append('console')

    # Применяем конфигурацию
    logging.config.dictConfig(logger_config)

# Настраиваем логирование
setup_logging()

def get_business_centers_in_district(district_id, district_name):
    """
    Функция для получения всех бизнес-центров в указанном районе
    """
    logger = logging.getLogger(__name__)
    all_business_centers = []
    page = 1
    page_size = CONFIG['data_collection']['page_size']
    max_pages = CONFIG['data_collection']['max_pages']

    logger.info(f"Начало сбора бизнес-центров для района: {district_name} (ID: {district_id})")

    while page <= max_pages:
        params = {
            'key': CONFIG['api']['key'],
            'q': 'бизнес-центр',
            'district_id': district_id,
            'type': 'branch',
            'page': page,
            'page_size': page_size,
            'fields': 'items.point,items.name,items.address_name,items.reviews'
        }

        url = "https://catalog.api.2gis.com/3.0/items"

        logger.debug(f"Запрос страницы {page} для района {district_name}")

        try:
            response = requests.get(url, params=params, timeout=CONFIG['api']['timeout'])
            logger.debug(f"Статус ответа: {response.status_code}")

            if response.status_code == 200:
                data = response.json()

                if 'result' in data and 'items' in data['result']:
                    business_centers = data['result']['items']

                    if business_centers:
                        logger.info(f"Загружено {len(business_centers)} бизнес-центров со страницы {page}")
                        for business_center in business_centers:
                            business_center['district_name'] = district_name
                        all_business_centers.extend(business_centers)
                    else:
                        logger.debug(f"Страница {page} пуста")
                else:
                    logger.warning(f"На странице {page} нет данных в ожидаемом формате")

                page += 1

            else:
                logger.error(f"HTTP ошибка: {response.status_code} для страницы {page}")
                page += 1

        except requests.exceptions.Timeout:
            logger.error(f"Таймаут запроса на странице {page}")
            page += 1
        except Exception as e:
            logger.error(f"Ошибка на странице {page}: {e}")
            page += 1

        if page <= max_pages:
            time.sleep(CONFIG['api']['sleep_between_requests'])

    logger.info(f"Завершение сбора для района {district_name}. Найдено бизнес-центров: {len(all_business_centers)}")
    return all_business_centers

def create_business_centers_dataset(all_business_centers):
    """
    Создает структурированный датасет из собранных данных о бизнес-центрах
    """
    logger = logging.getLogger(__name__)
    logger.info(f"Создание датасета из {len(all_business_centers)} бизнес-центров")

    data = []

    for business_center in all_business_centers:
        reviews_data = business_center.get('reviews', {})
        point_data = business_center.get('point', {})

        row = {
            'district': business_center.get('district_name', ''),
            'type': 'бизнес-центр',
            'name': business_center.get('name', ''),
            'address': business_center.get('address_name', ''),
            'latitude': point_data.get('lat', ''),
            'longitude': point_data.get('lon', ''),
            'rating': reviews_data.get('general_rating', 'нет рейтинга'),
            'reviews_count': reviews_data.get('general_review_count', 0),
            'ratings_count': reviews_data.get('general_review_count_with_stars', 0)
        }
        data.append(row)

    logger.info(f"Датасет бизнес-центров создан. Количество строк: {len(data)}")
    return pd.DataFrame(data)

# Основной код
if __name__ == "__main__":
    logger = logging.getLogger(__name__)

    logger.info("=" * 60)
    logger.info("ЗАПУСК СБОРА ДАННЫХ ПО БИЗНЕС-ЦЕНТРАМ МОСКВЫ")
    logger.info(f"Конфигурация логирования: уровень={CONFIG['logging']['level']}, файл={CONFIG['logging']['log_to_file']}, консоль={CONFIG['logging']['log_to_console']}")
    logger.info(f"Количество районов для обработки: {len(DISTRICT_IDS)}")
    logger.info("=" * 60)

    all_business_centers_moscow = []
    start_time = time.time()

    # Обрабатываем каждый район
    for i, (district_name, district_id) in enumerate(DISTRICT_IDS.items(), 1):
        logger.info(f"[{i}/{len(DISTRICT_IDS)}] Обработка района: {district_name}")

        try:
            business_centers_in_district = get_business_centers_in_district(district_id, district_name)
            all_business_centers_moscow.extend(business_centers_in_district)
            logger.info(f"Район '{district_name}' завершен. Бизнес-центров: {len(business_centers_in_district)}")

        except Exception as e:
            logger.error(f"Ошибка в районе {district_name}: {e}")
            continue

        if i < len(DISTRICT_IDS):
            time.sleep(CONFIG['data_collection']['sleep_between_districts'])

    # Финализация
    end_time = time.time()
    execution_time = end_time - start_time

    logger.info("=" * 60)
    logger.info(f"СБОР ДАННЫХ ЗАВЕРШЕН")
    logger.info(f"Время выполнения: {execution_time:.2f} секунд")
    logger.info(f"Всего собрано бизнес-центров: {len(all_business_centers_moscow)}")
    logger.info("=" * 60)

    # Создаем датасет
    if all_business_centers_moscow:
        df9 = create_business_centers_dataset(all_business_centers_moscow)
        filename = 'бизнес_центры_москвы_по_районам.csv'
        df9.to_csv(filename, index=False, encoding='utf-8')
        logger.info(f"Данные сохранены в {filename}")

        logger.info(f"СТАТИСТИКА: Всего бизнес-центров: {len(df9)}")
        logger.info(f"СТАТИСТИКА: Охвачено районов: {df9['district'].nunique()}")

    else:
        logger.error("Не удалось собрать данные о бизнес-центрах")
        print("Не удалось собрать данные о бизнес-центрах")

In [ ]:
df9

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Академический,бизнес-центр,"Ферро-Плаза, бизнес-центр","улица Кржижановского, 14 к3",55.679541,37.566590,3.7,8,30
1,Академический,бизнес-центр,"Микроэкономика, бизнес-центр","Большая Черёмушкинская улица, 34",55.679174,37.583232,3.7,5,36
2,Академический,бизнес-центр,"iCube, бизнес-центр","Нахимовский проспект, 58",55.683738,37.554129,4.7,11,36
3,Академический,бизнес-центр,"Академический, бизнес-центр","60-летия Октября проспект, 9 ст2",55.698817,37.580293,нет рейтинга,0,27
4,Академический,бизнес-центр,Бизнес-Недвижимость,"Новочерёмушкинская улица, 25",55.680746,37.578958,4.9,2,13
...,...,...,...,...,...,...,...,...,...
1409,Ярославский,бизнес-центр,"Контр-адмиралъ, бизнес-холл","Ярославское шоссе, 137",55.878985,37.720401,нет рейтинга,0,0
1410,Ясенево,бизнес-центр,"Маркет, бизнес-центр","Тарусская улица, 10",55.609361,37.533727,4.6,3,9
1411,Ясенево,бизнес-центр,Морбизнесцентр,"Профсоюзная улица, 125",55.621697,37.509935,нет рейтинга,0,0
1412,Ясенево,бизнес-центр,"Малый бизнес Москвы, центр услуг для бизнеса","Новоясеневский проспект, 1",55.619269,37.510106,5,1,1


In [ ]:
len(df9['район'].value_counts())

121

Не во всех районах Москвы есть бизнес-центры.

________________________________________________________________________________________________________________

##### Собираем все полученные данные в один датасет.

In [ ]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9], axis = 0)
df

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Академический,школа,Академическая школа №1534,"улица Гримау, 11а ст3",55.687755,37.582081,4.4,6,24
1,Академический,школа,"Лит, школа №1533","Профсоюзная улица, 24 к4",55.680605,37.562167,5,1,6
2,Академический,школа,"Академическая школа №1534, главный корпус","улица Кедрова, 11",55.684737,37.574167,4.1,4,21
3,Академический,школа,Школа №199,"улица Дмитрия Ульянова, 15 к1",55.690083,37.571609,5,1,6
4,Академический,школа,Школа №45,"улица Гримау, 8",55.691589,37.580152,4.5,2,10
...,...,...,...,...,...,...,...,...,...
1409,Ярославский,бизнес-центр,"Контр-адмиралъ, бизнес-холл","Ярославское шоссе, 137",55.878985,37.720401,нет рейтинга,0,0
1410,Ясенево,бизнес-центр,"Маркет, бизнес-центр","Тарусская улица, 10",55.609361,37.533727,4.6,3,9
1411,Ясенево,бизнес-центр,Морбизнесцентр,"Профсоюзная улица, 125",55.621697,37.509935,нет рейтинга,0,0
1412,Ясенево,бизнес-центр,"Малый бизнес Москвы, центр услуг для бизнеса","Новоясеневский проспект, 1",55.619269,37.510106,5,1,1


In [ ]:
df = df.sort_values(['район', 'тип'])
df

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
19804,Академический,бизнес-центр,"Ферро-Плаза, бизнес-центр","улица Кржижановского, 14 к3",55.679541,37.566590,3.7,8,30
19805,Академический,бизнес-центр,"Микроэкономика, бизнес-центр","Большая Черёмушкинская улица, 34",55.679174,37.583232,3.7,5,36
19806,Академический,бизнес-центр,"iCube, бизнес-центр","Нахимовский проспект, 58",55.683738,37.554129,4.7,11,36
19807,Академический,бизнес-центр,"Академический, бизнес-центр","60-летия Октября проспект, 9 ст2",55.698817,37.580293,нет рейтинга,0,27
19808,Академический,бизнес-центр,Бизнес-Недвижимость,"Новочерёмушкинская улица, 25",55.680746,37.578958,4.9,2,13
...,...,...,...,...,...,...,...,...,...
2471,Ясенево,школа,"Школа №1103, школа им. Героя РФ А.В. Соломатина","Литовский бульвар, 42 к2",55.608020,37.541734,4.9,1,4
2472,Ясенево,школа,Школа №2103,"Голубинская улица, 13 к2",55.603158,37.517547,нет рейтинга,0,2
2473,Ясенево,школа,"Школа им. Н.М. Карамзина, Остафьевский корпус №9","проезд Карамзина, 9 к2",55.591577,37.540696,нет рейтинга,0,0
2474,Ясенево,школа,"London Express, школа","Соловьиный проезд, 1",55.619451,37.543880,5,3,5


In [ ]:
df = df.reset_index()
del df['index']
df

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Академический,бизнес-центр,"Ферро-Плаза, бизнес-центр","улица Кржижановского, 14 к3",55.679541,37.566590,3.7,8,30
1,Академический,бизнес-центр,"Микроэкономика, бизнес-центр","Большая Черёмушкинская улица, 34",55.679174,37.583232,3.7,5,36
2,Академический,бизнес-центр,"iCube, бизнес-центр","Нахимовский проспект, 58",55.683738,37.554129,4.7,11,36
3,Академический,бизнес-центр,"Академический, бизнес-центр","60-летия Октября проспект, 9 ст2",55.698817,37.580293,нет рейтинга,0,27
4,Академический,бизнес-центр,Бизнес-Недвижимость,"Новочерёмушкинская улица, 25",55.680746,37.578958,4.9,2,13
...,...,...,...,...,...,...,...,...,...
21213,Ясенево,школа,"Школа №1103, школа им. Героя РФ А.В. Соломатина","Литовский бульвар, 42 к2",55.608020,37.541734,4.9,1,4
21214,Ясенево,школа,Школа №2103,"Голубинская улица, 13 к2",55.603158,37.517547,нет рейтинга,0,2
21215,Ясенево,школа,"Школа им. Н.М. Карамзина, Остафьевский корпус №9","проезд Карамзина, 9 к2",55.591577,37.540696,нет рейтинга,0,0
21216,Ясенево,школа,"London Express, школа","Соловьиный проезд, 1",55.619451,37.543880,5,3,5


In [ ]:
df.to_csv('итогAPI.csv', index=False, encoding='utf-8')

______________________________________________________________________________________________________________________________________________________

In [ ]:
dff = pd.read_csv('итогAPI.csv')

In [ ]:
dff

,район,тип,название,адрес,широта,долгота,рейтинг,количество_отзывов,количество_оценок
0,Академический,бизнес-центр,"Ферро-Плаза, бизнес-центр","улица Кржижановского, 14 к3",55.679541,37.566590,3.7,8,30
1,Академический,бизнес-центр,"Микроэкономика, бизнес-центр","Большая Черёмушкинская улица, 34",55.679174,37.583232,3.7,5,36
2,Академический,бизнес-центр,"iCube, бизнес-центр","Нахимовский проспект, 58",55.683738,37.554129,4.7,11,36
3,Академический,бизнес-центр,"Академический, бизнес-центр","60-летия Октября проспект, 9 ст2",55.698817,37.580293,нет рейтинга,0,27
4,Академический,бизнес-центр,Бизнес-Недвижимость,"Новочерёмушкинская улица, 25",55.680746,37.578958,4.9,2,13
...,...,...,...,...,...,...,...,...,...
21213,Ясенево,школа,"Школа №1103, школа им. Героя РФ А.В. Соломатина","Литовский бульвар, 42 к2",55.608020,37.541734,4.9,1,4
21214,Ясенево,школа,Школа №2103,"Голубинская улица, 13 к2",55.603158,37.517547,нет рейтинга,0,2
21215,Ясенево,школа,"Школа им. Н.М. Карамзина, Остафьевский корпус №9","проезд Карамзина, 9 к2",55.591577,37.540696,нет рейтинга,0,0
21216,Ясенево,школа,"London Express, школа","Соловьиный проезд, 1",55.619451,37.543880,5,3,5


Сгруппируем данные полученной после апи-парсинга таблицы, чтобы в ней содержалось количсетво всех объектов инфраструктуры для каждого района Москвы.  

In [ ]:
types_count = pd.crosstab(index=dff['район'], columns=dff['тип'])
types_count

тип,бизнес-центр,детский сад,кафе,магазин,музей,парк,поликлиника,торговый центр,школа
район,,,,,,,,,
Академический,13,30,38,50,8,1,9,2,19
Алексеевский,17,21,44,50,2,0,9,3,16
Алтуфьевский,7,10,5,32,1,1,4,2,6
Арбат,27,11,50,50,27,1,5,8,9
Аэропорт,21,32,50,50,7,12,9,7,15
...,...,...,...,...,...,...,...,...,...
Южное Тушино,11,19,24,50,0,2,3,4,22
Южнопортовый,18,19,50,50,1,8,10,9,15
Якиманка,26,9,50,47,19,3,6,4,13


In [ ]:
types_count.to_csv('количество.csv')

______________________________________________________________________________________________________________________________________________________